In [21]:
# import the relevant libraries
import datetime
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime
from pptx import Presentation
from pptx.util import Inches
from pptx.util import Pt
from pptx.enum.text import PP_ALIGN
from datetime import date
from pptx.dml.color import RGBColor
import os
import sys
from scipy import stats

# Import the required tkinter Libraries
import tkinter as tk
from tkinter import *
from tkinter import ttk, filedialog

#from tkinter.filedialog import askopenfile
from tkinter import messagebox
from threading import Thread
import zipfile

##Image libraries
import plotly.io as pio
import tempfile
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

In [24]:
# input cell

# running_cells = ['L2','L1'] #['U1', 'U2', 'L2', 'L1']

# name_of_stack = ['91-56']# manual input

# stack_dictionary = {'U1':'', 'U2':'', 'L2':'', 'L1':'91-56'}

# stack_dictionary = {'U1':'', 'U2':'', 'L2':'96-56', 'L1':'93-56'}

template_ppt_fn = 'ckb_template_U0002'
# nptc template fn = 'ntpc_template'
# ckb template fn = 'ckb_template_U0002'
# ckb template fn = 'ckb_template_U05'

ppt_fn = '2023-Aug-1_1200hrs_IST_SPR0001_update' # change date only
# nptc otput fn = '2023-May-09_0200hrs_IST_SPR0001_update'
# ckb output fn = '2023-May-09_0200hrs_IST_Unit0002_update'

folder_path = "./extracted_files"
csv_file_list = os.listdir("./extracted_files/")
print(csv_file_list)
path = "./extracted_files/"
image_folder_name = './images'
sys_data = pd.concat([pd.read_csv(path + f, error_bad_lines=False) for f in csv_file_list ], ignore_index=True)

['DATA_U7_2023_07_08.csv', 'DATA_U7_2023_07_10.csv', 'DATA_U7_2023_07_11.csv', 'DATA_U7_2023_07_12.csv', 'DATA_U7_2023_07_13.csv', 'DATA_U7_2023_07_14.csv', 'DATA_U7_2023_07_15.csv', 'DATA_U7_2023_07_16.csv', 'DATA_U7_2023_07_17.csv', 'DATA_U7_2023_07_18.csv', 'DATA_U7_2023_07_19.csv', 'DATA_U7_2023_07_20.csv', 'DATA_U7_2023_07_21.csv', 'DATA_U7_2023_07_24.csv', 'DATA_U7_2023_07_25.csv', 'DATA_U7_2023_07_26.csv', 'DATA_U7_2023_07_27.csv', 'DATA_U7_2023_07_28.csv', 'DATA_U7_2023_07_29.csv', 'DATA_U7_2023_07_30.csv', 'DATA_U7_2023_07_31.csv']


Skipping line 541: expected 345 fields, saw 369
Skipping line 542: expected 345 fields, saw 369
Skipping line 543: expected 345 fields, saw 369
Skipping line 544: expected 345 fields, saw 369
Skipping line 545: expected 345 fields, saw 369
Skipping line 546: expected 345 fields, saw 369
Skipping line 547: expected 345 fields, saw 369
Skipping line 548: expected 345 fields, saw 369
Skipping line 549: expected 345 fields, saw 369
Skipping line 550: expected 345 fields, saw 369
Skipping line 551: expected 345 fields, saw 369
Skipping line 552: expected 345 fields, saw 369
Skipping line 553: expected 345 fields, saw 369
Skipping line 554: expected 345 fields, saw 369
Skipping line 555: expected 345 fields, saw 369
Skipping line 556: expected 345 fields, saw 369
Skipping line 557: expected 345 fields, saw 369
Skipping line 558: expected 345 fields, saw 369
Skipping line 559: expected 345 fields, saw 369
Skipping line 560: expected 345 fields, saw 369
Skipping line 561: expected 345 fields, 

In [3]:
#Calculating run hrs by considering timegap more than 10 sec as an interruption.

# converting datetime from string to datetime format
sys_data['Time'] = pd.to_datetime(sys_data['Time'],dayfirst=True)
## initialising run_hrs and a duration list
run_hrs = 0
duration_hr = [0.000]

## definig the run hrs completed by defining delta(s)    
for i in range (1,len(sys_data)):
    s = (sys_data['Time'][i] - sys_data['Time'][i-1]).seconds
    if s <= 300 :
        run_hrs = run_hrs + (s/3600)
        duration_hr.append(run_hrs)
    elif s > 300:
        run_hrs = run_hrs + (10/3600)
        duration_hr.append(run_hrs)
#adding a column for duration in dataframe
sys_data['duration_hr'] = duration_hr        
# sys_data[['Time','duration_hr']].head()
# Output the total run duration in round figure in hrs (excluding interruptions)

total_duration_hrs = round(max(sys_data['duration_hr']),0)
print("Total Run Duration is "+ '"'+str(int(total_duration_hrs))+" hours"+'" (excluding interruptions)')
Start_time = str(min(sys_data['Time']))
End_time = str(max(sys_data['Time']))

print("From : "+Start_time)
print("To   : "+End_time)

# replace the the string '-NA-' in data with blank
sys_data = sys_data.replace('-NA-',np.nan)

#remove rows that have any blank values
# sys_data = sys_data.dropna(axis = 0)

Total Run Duration is "339 hours" (excluding interruptions)
From : 2023-07-08 00:00:05
To   : 2023-07-31 14:41:40


In [4]:
# Vcell_data[Vcell_data['Date'] >= '2023-06-18 12:06:20'][['ASR_Stack4']]

In [5]:
# Vcell_data[Vcell_data['Date'] >= '2023-06-16 08:51:55'][[
#                                                          'Max_V_C13_1-C16_16'
# #                                                          ,'ASR_Stack4'
# ]].max()

In [6]:
## pre-processing the data ##
## remove the blank cols from data
sys_data_empty = []
for i in sys_data.columns:
    if len(sys_data[i].value_counts()) == 0:
        sys_data_empty.append(i)
        sys_data = sys_data.drop(i,axis = 1)        
# print("The following fields are completely blank in the data, hence are removed & stored separately:", sys_data_empty)

#convert the required fields from object to numeric datatype
obj_cols = sys_data.loc[:, ~sys_data.columns.isin(['Time', 'MP State', 'MS State','Train 1 State', 'Train 2 State', 'PMP 102'])].columns
sys_data[obj_cols] = sys_data[obj_cols].apply(pd.to_numeric, errors='coerce', axis=1)
#make all the col names in uppercase
sys_data.columns = sys_data.columns.str.upper()
#rename TIME col as Date
sys_data.rename(columns= {'TIME':'Date'},inplace = True)

In [7]:
Vcell_data = sys_data
# make a dict containing the positions of the last of the 56 cells
positions = {'U1': 4, 'U2': 8, 'L2': 12, 'L1': 16}
# loop through this dict
for position, cell in positions.items():
    # drop the unused cells (57-64)
    Vcell_data.drop(columns=['C{}-{}'.format(cell, i) for i in range(9,17)], inplace=True)
    # get the known start and end cells (1-56)
    start = 'C{}-1'.format(str(cell-3))
    end = 'C{}-8'.format(str(cell))
    # define the active condition for the columns
    active_condition = (Vcell_data['MP STATE'] == 'STACK PWR') & (Vcell_data['MS STATE'] == 'ARMED')
    # take a temp df where the active condition is satisfied and get the time and current
    temp = Vcell_data.loc[active_condition, ['Date', 'STACK CURRENT']]
    # calculate the median of the cell voltages (some times voltages are strings so need to convert to floats)
    temp['{} median'.format(position)] = Vcell_data.loc[active_condition,start:end].astype('float64').median(axis=1, numeric_only=True)
    # calculate the mean of the cell voltages
    temp['{} mean'.format(position)] = Vcell_data.loc[active_condition,start:end].astype('float64').mean(axis=1, numeric_only=True)
    # calculate the min of the cell voltages
    temp['{} min'.format(position)] = Vcell_data.loc[active_condition,start:end].astype('float64').min(axis=1, numeric_only=True)
    # calculate the max of the cell voltages
    temp['{} max'.format(position)] = Vcell_data.loc[active_condition,start:end].astype('float64').max(axis=1, numeric_only=True)
    # calculate the standard deviation of the cell voltages
    temp['{} std'.format(position)] = Vcell_data.loc[active_condition,start:end].astype('float64').std(axis=1, numeric_only=True)
#     print(temp.head())
#     make a column that indicates if a stack is present in the current position based on
#     if the median voltage is greater than 1.4 when the current is > 9
    temp['{} present'.format(position)] = (temp['{} median'.format(position)] > 1.4) | ((temp['{} median'.format(position)] > 1.4) & (temp['STACK CURRENT'] >= 9))
#     print(temp.head())
    
    # update the stack present column to be nan whenever the current is less than 9
    temp.loc[((temp['STACK CURRENT'] < 9) & (temp['{} present'.format(position)] == False)), '{} present'.format(position)] = np.nan
#     temp.to_csv(position + '.csv')
    
    # drop the time and stack current columns before joining the temp df with the original df
    temp.drop(columns=['Date', 'STACK CURRENT'], inplace=True)
    # join the temp df where True with the original df
    Vcell_data = pd.concat([Vcell_data, temp], axis = 1)


In [8]:
List_C1_1_C4_16 = [
    'C1-1', 'C1-2', 'C1-3', 'C1-4', 'C1-5', 'C1-6', 'C1-7', 'C1-8', 'C1-9', 'C1-10', 'C1-11', 'C1-12', 'C1-13', 'C1-14', 'C1-15', 'C1-16', 'C2-1', 'C2-2', 'C2-3', 'C2-4', 'C2-5', 'C2-6', 'C2-7', 'C2-8', 'C2-9', 'C2-10', 'C2-11', 'C2-12', 'C2-13', 'C2-14', 'C2-15', 'C2-16', 'C3-1', 'C3-2', 'C3-3', 'C3-4', 'C3-5', 'C3-6', 'C3-7', 'C3-8', 'C3-9', 'C3-10', 'C3-11', 'C3-12', 'C3-13', 'C3-14', 'C3-15', 'C3-16', 'C4-1', 'C4-2', 'C4-3', 'C4-4', 'C4-5', 'C4-6', 'C4-7', 'C4-8'
# #                    ,'C4-9', 'C4-10', 'C4-11', 'C4-12', 'C4-13', 'C4-14', 'C4-15', 'C4-16'
                  ]


List_C5_1_C8_16 = [
'C5-1', 'C5-2', 'C5-3', 'C5-4', 'C5-5', 'C5-6', 'C5-7', 'C5-8', 'C5-9', 'C5-10', 'C5-11', 'C5-12', 'C5-13', 'C5-14', 'C5-15', 'C5-16', 'C6-1', 'C6-2', 'C6-3', 'C6-4', 'C6-5', 'C6-6', 'C6-7', 'C6-8', 'C6-9', 'C6-10', 'C6-11', 'C6-12', 'C6-13', 'C6-14', 'C6-15', 'C6-16', 'C7-1', 'C7-2', 'C7-3', 'C7-4', 'C7-5', 'C7-6', 'C7-7', 'C7-8', 'C7-9', 'C7-10', 'C7-11', 'C7-12', 'C7-13', 'C7-14', 'C7-15', 'C7-16'       
                   ,'C8-1', 'C8-2', 'C8-3', 'C8-4', 'C8-5', 'C8-6', 'C8-7', 'C8-8'
#                    , 'C8-9', 'C8-10', 'C8-11', 'C8-12', 'C8-13', 'C8-14', 'C8-15', 'C8-16'
                  ]

List_C9_1_C12_16 = [
    'C9-1', 'C9-2', 'C9-3', 'C9-4', 'C9-5', 'C9-6', 'C9-7', 'C9-8', 'C9-9', 'C9-10', 'C9-11', 'C9-12', 'C9-13', 'C9-14', 'C9-15', 'C9-16', 'C10-1', 'C10-2', 'C10-3', 'C10-4', 'C10-5', 'C10-6', 'C10-7', 'C10-8', 'C10-9', 'C10-10', 'C10-11', 'C10-12', 'C10-13', 'C10-14', 'C10-15', 'C10-16', 'C11-1', 'C11-2', 'C11-3', 'C11-4', 'C11-5', 'C11-6', 'C11-7', 'C11-8', 'C11-9', 'C11-10', 'C11-11', 'C11-12', 'C11-13', 'C11-14', 'C11-15', 'C11-16', 'C12-1', 'C12-2', 'C12-3', 'C12-4', 'C12-5', 'C12-6', 'C12-7', 'C12-8'
#                    , 'C12-9', 'C12-10', 'C12-11', 'C12-12', 'C12-13', 'C12-14', 'C12-15', 'C12-16'
                  ]

List_C13_1_C16_16 = [
    'C13-1', 'C13-2', 'C13-3', 'C13-4', 'C13-5', 'C13-6', 'C13-7', 'C13-8', 'C13-9', 'C13-10', 'C13-11', 'C13-12', 'C13-13', 'C13-14', 'C13-15', 'C13-16', 'C14-1', 'C14-2', 'C14-3', 'C14-4', 'C14-5', 'C14-6', 'C14-7', 'C14-8', 'C14-9', 'C14-10', 'C14-11', 'C14-12', 'C14-13', 'C14-14', 'C14-15', 'C14-16', 'C15-1', 'C15-2', 'C15-3', 'C15-4', 'C15-5', 'C15-6', 'C15-7', 'C15-8', 'C15-9', 'C15-10', 'C15-11', 'C15-12', 'C15-13', 'C15-14', 'C15-15', 'C15-16', 'C16-1', 'C16-2', 'C16-3', 'C16-4', 'C16-5', 'C16-6', 'C16-7', 'C16-8'
#                     , 'C16-9', 'C16-10', 'C16-11', 'C16-12', 'C16-13', 'C16-14', 'C16-15', 'C16-16'
                   ]


In [9]:
## making all stack cells cols value to nan where stack present column is false or nan
cell_range = {'U1':List_C1_1_C4_16, 'U2':List_C5_1_C8_16, 'L2':List_C9_1_C12_16, 'L1':List_C13_1_C16_16}
for position , cells in cell_range.items():
    Vcell_data.loc[((Vcell_data['{} present'.format(position)].isna()) | (Vcell_data['{} present'.format(position)] == False)), cells] = np.nan


In [10]:
Vcell_data['Avg_V_C1_1-C4_16'] = (Vcell_data[List_C1_1_C4_16].sum(axis=1))/len(List_C1_1_C4_16)
Vcell_data['Max_V_C1_1-C4_16'] = Vcell_data[List_C1_1_C4_16].max(axis=1) 
Vcell_data['Min_V_C1_1-C4_16'] = Vcell_data[List_C1_1_C4_16].min(axis=1)

Vcell_data['Avg_V_C5_1-C8_16'] = (Vcell_data[List_C5_1_C8_16].sum(axis=1))/len(List_C5_1_C8_16)
Vcell_data['Max_V_C5_1-C8_16'] = Vcell_data[List_C5_1_C8_16].max(axis=1)
Vcell_data['Min_V_C5_1-C8_16'] = Vcell_data[List_C5_1_C8_16].min(axis=1)

Vcell_data['Avg_V_C9_1-C12_16'] = (Vcell_data[List_C9_1_C12_16].sum(axis=1))/len(List_C9_1_C12_16)
Vcell_data['Max_V_C9_1-C12_16'] = Vcell_data[List_C9_1_C12_16].max(axis=1)
Vcell_data['Min_V_C9_1-C12_16'] = Vcell_data[List_C9_1_C12_16].min(axis=1)   

Vcell_data['Avg_V_C13_1-C16_16'] = (Vcell_data[List_C13_1_C16_16].sum(axis=1))/len(List_C13_1_C16_16)
Vcell_data['Max_V_C13_1-C16_16'] = Vcell_data[List_C13_1_C16_16].max(axis=1)
Vcell_data['Min_V_C13_1-C16_16'] = Vcell_data[List_C13_1_C16_16].min(axis=1) 

In [11]:
## making cd, voltage charts., and ASR charts with dataframe created - Vcell_data
Vcell_data['CURRENT DENSITY'] = Vcell_data['STACK CURRENT'] / 702

#ASR in mohm-cm2
Vcell_data['ASR_Stack1'] = (Vcell_data['Avg_V_C1_1-C4_16']/Vcell_data['CURRENT DENSITY'])*1000
Vcell_data['ASR_Stack2'] = (Vcell_data['Avg_V_C5_1-C8_16']/Vcell_data['CURRENT DENSITY'])*1000
Vcell_data['ASR_Stack3'] = (Vcell_data['Avg_V_C9_1-C12_16']/Vcell_data['CURRENT DENSITY'])*1000
Vcell_data['ASR_Stack4'] = (Vcell_data['Avg_V_C13_1-C16_16']/Vcell_data['CURRENT DENSITY'])*1000

Vcell_data['ASR(max)_Stack1'] = (Vcell_data['Max_V_C1_1-C4_16']/Vcell_data['CURRENT DENSITY'])*1000
Vcell_data['ASR(max)_Stack2'] = (Vcell_data['Max_V_C5_1-C8_16']/Vcell_data['CURRENT DENSITY'])*1000 
Vcell_data['ASR(max)_Stack3'] = (Vcell_data['Max_V_C9_1-C12_16']/Vcell_data['CURRENT DENSITY'])*1000
Vcell_data['ASR(max)_Stack4'] = (Vcell_data['Max_V_C13_1-C16_16']/Vcell_data['CURRENT DENSITY'])*1000

Current_Density = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['CURRENT DENSITY'],
                       mode = 'markers',
                       marker=dict(color='#2ca02c'),
                       showlegend = True,
#                        mode = 'lines',
                       name = 'Current Density (A/c'+'m\u00b2)',
                       yaxis = 'y2'
                       #text = df.column (this is the info displayed on hover)
                       )

# update the data list
data_plot = [Current_Density]

# configure the layout
layout = go.Layout(
            title = 'Current Density',
            #to define axis, can either use go.layout.etc or dict(etc)
            xaxis = go.layout.XAxis(
#                                     title = 'Date',
                                    tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                    hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                    showgrid = False,
                                    #range=[100,steady_state['Duration(hr)'].max()]
                                    ),    
            yaxis= dict(title='Current Density'+'<br>'+'(A/c'+'m\u00b2)',
#                                  overlaying='y',
                                 side = 'right',
                                 showgrid = False,
                                 range = [0,6],
                                 rangemode='tozero'),
            legend= dict(xanchor="right",x=1.2,yanchor="top", y =1.2, bgcolor = 'rgba(0,0,0,0)'),
            template = 'simple_white',
            font = dict(size = 18)
            )


# create the figure
fig0 = go.Figure(data = data_plot, layout = layout)
fig0.update_xaxes(tickangle=90)

fig0.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
       yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
       yaxis2 = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) 
                 )


fig0.write_image('{}/Current_Density.png'.format(image_folder_name), engine="orca")
fig0.write_html('{}/Current_Density.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# fig0.show()

In [12]:
#creating Avg_voltage_chart
Avg_StackU1 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['Avg_V_C1_1-C4_16'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack U1 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
Avg_StackU2 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['Avg_V_C5_1-C8_16'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack U2 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
Avg_StackL2 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['Avg_V_C9_1-C12_16'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack L2 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
Avg_StackL1 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['Avg_V_C13_1-C16_16'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack L1 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
Current_Density = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['CURRENT DENSITY'],
                       mode = 'markers',
#                        mode = 'lines',
                       name = 'Current Density (A/c'+'m\u00b2)',
                       yaxis = 'y2'
                       #text = df.column (this is the info displayed on hover)
                       )
           
# update the data list
data = [
    Avg_StackU1,
    Avg_StackU2,
    Avg_StackL2,
    Avg_StackL1,
    Current_Density]

# configure the layout
layout = go.Layout(
            title = 'LHC Avg Cell Voltage'+'<br>'+ '& Current Density',
            #to define axis, can either use go.layout.etc or dict(etc)
            xaxis = go.layout.XAxis(
#                                     title = 'Date',
                                    tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                    hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                    showgrid = False,
                                    #range=[100,steady_state['Duration(hr)'].max()]
                                    ),    
            yaxis = dict(title='Avg Voltage (V)',
#                                 font = 15,
                                showgrid = False,
                                range=[1.3, 2.3],
                                rangemode='tozero'),
            yaxis2= dict(title='Current Density'+'<br>'+'(A/c'+'m\u00b2)',
                                 overlaying='y',
                                 side = 'right',
                                 showgrid = False,
                                 range = [0,2],
                                 rangemode='tozero'),
            legend= dict(xanchor="right",x=1.5,yanchor="top", y =2, bgcolor = 'rgba(0,0,0,0)'),
            template = 'simple_white',
            font = dict(size = 18)
            )


# create the figure
fig5 = go.Figure(data = data, layout = layout)
fig5.update_xaxes(tickangle=90)

fig5.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
       yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
       yaxis2 = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) 
                 )


fig5.write_image('{}/LHC_Avg_Stack_Voltage_&_Current_Density.png'.format(image_folder_name), engine="orca")
fig5.write_html('{}/LHC_Avg_Stack_Voltage_&_Current_Density.html'.format(image_folder_name), include_plotlyjs=('cdn'))
# fig5.show()

# creating Max_Voltage & Current Density Chart
Max_StackU1 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['Max_V_C1_1-C4_16'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack U1 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
Max_StackU2 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['Max_V_C5_1-C8_16'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack U2 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
Max_StackL2 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['Max_V_C9_1-C12_16'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack L2 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
Max_StackL1 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['Max_V_C13_1-C16_16'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack L1 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
Current_Density = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['CURRENT DENSITY'],
                       mode = 'markers',
#                        mode = 'lines',
                       name = 'Current Density (A/c'+'m\u00b2)',
                       yaxis = 'y2'
                       #text = df.column (this is the info displayed on hover)
                       )
           
# update the data list
data = [
    Max_StackU1,
    Max_StackU2,
    Max_StackL2,
    Max_StackL1,
    Current_Density]

# configure the layout
layout = go.Layout(
            title = 'LHC Max Cell Voltage'+'<br>'+'& Current Density',
            #to define axis, can either use go.layout.etc or dict(etc)
            xaxis = go.layout.XAxis(
#                                     title = 'Date',
                                    tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                    hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                    showgrid = False,
                                    #range=[100,steady_state['Duration(hr)'].max()]
                                    ),    
            yaxis = dict(title='Max Voltage (V)',
#                                 font = 15,
                                range=[1.3, 2.3],
                                showgrid = False,
                                rangemode='tozero'),
            yaxis2= dict(title='Current Density'+'<br>'+'(A/c'+'m\u00b2)',
                                 overlaying='y',
                                 range = [0,2],
                                 side = 'right',
                                 showgrid = False,
                                 rangemode='tozero'),#,
#                                  range=[0,600]),
            legend= dict(xanchor="right",x=1.5,yanchor="top", y =2, bgcolor = 'rgba(0,0,0,0)'),
            template = 'simple_white',
            font = dict(size = 18)
            )

# create the figure
fig6 = go.Figure(data = data, layout = layout)
fig6.update_xaxes(tickangle=90)

fig6.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
       yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
       yaxis2 = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) 
                 )

fig6.write_image('{}/LHC_Max_Stack_Voltage_&_Current_Density.png'.format(image_folder_name), engine="orca")
fig6.write_html('{}/LHC_Max_Stack_Voltage_&_Current_Density.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# fig6.show()

# create min voltage and cd chart
Min_StackU1 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['Min_V_C1_1-C4_16'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack U2 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
Min_StackU2 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['Min_V_C5_1-C8_16'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack U2 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
Min_StackL2 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['Min_V_C9_1-C12_16'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack L2 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
Min_StackL1 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['Min_V_C13_1-C16_16'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack L1 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )   #text = df.column (this is the info displayed on hover)
Current_Density = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['CURRENT DENSITY'],
                       mode = 'markers',
#                        mode = 'lines',
                       name = 'Current Density (A/c'+'m\u00b2)',
                       yaxis = 'y2'
                       #text = df.column (this is the info displayed on hover)
                       )                       

# update the data list
data = [
    Min_StackU1,
    Min_StackU2, 
    Min_StackL2,
    Min_StackL1,
    Current_Density]

# configure the layout
layout = go.Layout(
            title = 'LHC Min Cell Voltage'+'<br>'+'& Current Density',
            #to define axis, can either use go.layout.etc or dict(etc)
            xaxis = go.layout.XAxis(
#                                     title = 'Date',
                                    tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                    hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                    showgrid = False,
                                    #range=[100,steady_state['Duration(hr)'].max()]
                                    ),    
            yaxis = dict(title='Min Voltage (V)',
#                                 font = 15,
                                range = [1.3, 2.3],
                                showgrid = False,
                                rangemode='tozero'),
            yaxis2= dict(title='Current Density'+'<br>'+'(A/c'+'m\u00b2)',
                                 range = [0,2],
                                 overlaying='y',
                                 side = 'right',
                                 showgrid = False,
                                 rangemode='tozero'),#,
#                                  range=[0,600]),
            legend= dict(xanchor="right",x=1.5,yanchor="top", y =2, bgcolor = 'rgba(0,0,0,0)'),
            template = 'simple_white',
            font = dict(size = 18)
            )

# create the figure
fig7 = go.Figure(data = data, layout = layout)
fig7.update_xaxes(tickangle=90)

fig7.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
       yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
       yaxis2 = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) 
                 )

fig7.write_image('{}/LHC_Min_Stack_Voltage_&_Current_Density.png'.format(image_folder_name), engine="orca")
fig7.write_html('{}/LHC_Min_Stack_Voltage_&_Current_Density.html'.format(image_folder_name), include_plotlyjs=('cdn'))
# fig7.show()

# create ASR Analysis Chart
ASR_StackU1 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['ASR_Stack1'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack U1 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
ASR_StackU2 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['ASR_Stack2'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack U2 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
ASR_StackL2 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['ASR_Stack3'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack L2 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
ASR_StackL1 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['ASR_Stack4'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack L1 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )

# # update the data list
data = [
    ASR_StackU1,
    ASR_StackU2,
    ASR_StackL2,
    ASR_StackL1, 
    Current_Density]

# configure the layout
layout = go.Layout(
            title = 'Stack ASR Chart',
            #to define axis, can either use go.layout.etc or dict(etc)
            xaxis = go.layout.XAxis(
#                                     title = 'Date',
                                    tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                    hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                    showgrid = False,
                                    #range=[100,steady_state['Duration(hr)'].max()]
                                    ),    
            yaxis = dict(title='ASR (mΩ-c'+'m\u00b2)',
#                                 font = 15,
                                range = [0,15000],
                                showgrid = False,
                                rangemode='tozero'),
            yaxis2= dict(title='Current Density'+'<br>'+'(A/c'+'m\u00b2)',
                                 overlaying='y',
                                 side = 'right',
                                 showgrid = False,
                                 rangemode='tozero',
                                 range=[0,1.5]),
            legend= dict(xanchor="right",x=1.5,yanchor="top", y=2, bgcolor = 'rgba(0,0,0,0)'),
            template = 'simple_white',
            font = dict(size = 18)
            )

# create the figure
fig8 = go.Figure(data = data, layout = layout)

fig8.update_xaxes(tickangle=90)

fig8.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
       yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
       yaxis2 = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) 
                 ) 

fig8.write_image('{}/Stack_ASR_Chart.png'.format(image_folder_name), engine="orca")
fig8.write_html('{}/Stack_ASR_Chart.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# fig8.show()

#create Max ASR chart
max_ASR_StackU1 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['ASR(max)_Stack1'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack U1 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
max_ASR_StackU2 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['ASR(max)_Stack2'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack U2 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
max_ASR_StackL2 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['ASR(max)_Stack3'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack L2 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
max_ASR_StackL1 = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['ASR(max)_Stack4'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'Stack L1 (V)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )

# update the data list
data = [
    max_ASR_StackU1,
    max_ASR_StackU2,
    max_ASR_StackL2,
    max_ASR_StackL1,
    Current_Density]

# configure the layout
layout = go.Layout(
            title = 'Stack ASR (max) Chart',
            #to define axis, can either use go.layout.etc or dict(etc)
            xaxis = go.layout.XAxis(
#                                     title = 'Date',
                                    tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                    hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                    showgrid = False,
                                    #range=[100,steady_state['Duration(hr)'].max()]
                                    ),    
            yaxis = dict(title='Max ASR (mΩ-c'+'m\u00b2)',
                                range = [0,15000],
#                                 font = 15,
                                showgrid = False,
                                rangemode='tozero'),
            yaxis2= dict(title='Current Density'+'<br>'+ '(A/cm\u00b2)',
                                range = [0,1.5],
                                 overlaying='y',
                                 side = 'right',
                                 showgrid = False,
                                 rangemode='tozero'),
#                                  range=[0,600]),
            legend= dict(xanchor="right",x=1.5,yanchor="top", y =2, bgcolor = 'rgba(0,0,0,0)'),
            template = 'simple_white',
            font = dict(size = 18)
            )

# create the figure
fig9 = go.Figure(data = data, layout = layout)

fig9.update_xaxes(tickangle=90)

fig9.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
       yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
       yaxis2 = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) 
                 ) 

fig9.write_image('{}/Stack_ASR(max)_Chart.png'.format(image_folder_name), engine="orca")
fig9.write_html('{}/Stack_ASR(max)_Chart.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# fig9.show()

In [13]:
# set upper and lower limits of data
sys_data['CURRENT DENSITY'] = sys_data['STACK CURRENT'] / 702

Col_List_0to100 = ['PRT 702','PRT 703','PRT 704','PRT 705','PRT 706','KTC 901','HYS 101' ,'HYS 401' ,'HYS 501' ,'HYS 102' ,'OXS 101' ,'LVL 101', 'TTC 102' ,'PRT 401' ,'PRT 402' ,'KTC 401' ,'TE 401' ,'TTC 101' ,'TTC 301' ,'PRT 101']
condition = (sys_data[Col_List_0to100] >= 0) & (sys_data[Col_List_0to100] <= 100)
sys_data = sys_data[condition.all(axis=1)]

sys_data = sys_data[sys_data['PRT 102'] >= 0]
# sys_data = sys_data[sys_data['DPT 401'] >= -70]

Col_List_0to200 = ['COS 701','COS 702','COS 101']
condition = (sys_data[Col_List_0to200] >= 0) & (sys_data[Col_List_0to200] <= 200)
sys_data = sys_data[condition.all(axis=1)]

#create pressure and conductivity chart
Pressure_102 = go.Scattergl(
                       x = sys_data['Date'],
                       y = sys_data['PRT 102'],
                       mode = 'markers',
                       #mode = 'lines',
                       name = 'PRT 102 (bar)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
Pressure_401 = go.Scattergl(
                       x = sys_data['Date'],
                       y = sys_data['PRT 401'],
                       mode = 'markers',
                       #mode = lines,
                       name = 'PRT 401 (bar)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
Conductivity_101 = go.Scattergl(
                        x = sys_data['Date'],
                        y = sys_data['COS 101'],
                        mode = 'markers',
                        #mode = 'lines',
                        name = 'COS 101 (µS/cm)',
                        yaxis = 'y2',
                        #text = df.column (this is the info displayed on hover)
                        )
# update the data list
data = [Pressure_102, Pressure_401, Conductivity_101]

# configure the layout
layout = go.Layout(
            title = 'LHC Pressure &'+'<br>'+'Conductivity chart',
            #to define axis, can either use go.layout.etc or dict(etc)
            xaxis = go.layout.XAxis(
#                                     title = 'Date',
                                    tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                    hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                    showgrid = False,
                                    #range=[100,steady_state['Duration(hr)'].max()]
                                    ),
            yaxis = dict(title='Pressure (bar)',
#                                 font = 15,
                                showgrid = False,
                                rangemode='tozero'),
            yaxis2= dict(title='COS 101 (µS/cm)',
#                                  overlaying='y',
                                 side = 'right',
                                 showgrid = False,
                                 rangemode='tozero',
#                                  range=[0,12.5]
                        ),
          legend= dict(xanchor="right",x=1.3, yanchor="top", y =1.5, bgcolor = 'rgba(0,0,0,0)'),
          template = 'simple_white',
          font = dict(size = 18)
              )
# create the figure
fig1 = go.Figure(data = data, layout = layout)

fig1.update_xaxes(tickangle=90)

fig1.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) )

fig1.write_image('{}/LHC_Pressure_&_Conductivity_Chart.png'.format(image_folder_name), engine="orca")
fig1.write_html('{}/LHC_Pressure_&_Conductivity_Chart.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# fig1.show()

# create TTC,KTC & LVL sensors chart
list_cols = ['TTC 101','TTC 102', 'TTC 301','KTC 401','LVL 101']
condition = (sys_data[list_cols] <= 100)
sys_data = sys_data[condition.all(axis=1)]

TTC101 = go.Scattergl(
                       x = sys_data['Date'],
                       y = sys_data['TTC 101'],
                       mode = 'markers',
                       #marker=dict(size=3),
                       #mode = 'lines',
                       name = 'TTC 101 (°C)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
TTC102 = go.Scattergl(
                       x = sys_data['Date'],
                       y = sys_data['TTC 102'],
                       mode = 'markers',
                       #marker=dict(size=3),
                       #mode = lines,
                       name = 'TTC 102 (°C)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
TTC301  = go.Scattergl(
                        x = sys_data['Date'],
                        y = sys_data['TTC 301'],
                        mode = 'markers',
                        #marker=dict(size=3),
                        #mode = 'lines',
                        name = 'TTC 301 (°C)',
                        yaxis = 'y1',
                        #text = df.column (this is the info displayed on hover)
                        )
KTC401  = go.Scattergl(
                        x = sys_data['Date'],
                        y = sys_data['KTC 401'],
                        mode = 'markers',
                        #marker=dict(size=3),
                        #mode = 'lines',
                        name = 'KTC 401 (°C)',
                        yaxis = 'y1',
                        #text = df.column (this is the info displayed on hover)
                        )
LVL101  = go.Scattergl(
                        x = sys_data['Date'],
                        y = sys_data['LVL 101'],
                        mode = 'markers',
                        #marker=dict(size=3),
                        #mode = 'lines',
                        name = 'LVL 101 (%)',
                        yaxis = 'y2',
                        #text = df.column (this is the info displayed on hover)
                        ) 



# update the data list
data = [TTC101, TTC102, TTC301, KTC401, LVL101]

# configure the layout
layout = go.Layout(
            title = 'LHC Temp &'+'<br>'+'Level sensor chart',
            #to define axis, can either use go.layout.etc or dict(etc)
            xaxis = go.layout.XAxis(
#                                     title = 'Date',
                                    tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                    hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                    showgrid = False,
                                    #range=[100,steady_state['Duration(hr)'].max()]
                                    ),
    
            yaxis = dict(title='Temp(°C),',
                                showgrid = False,
                                range = [0,100],
                                rangemode='tozero'),
            yaxis2= dict(title='Water level (%)',
                                 overlaying='y',
                                 side = 'right',
                                 showgrid = False,
                                 rangemode='tozero',
                                 range=[0,100]),
            legend= dict(xanchor="right",x=1.3,yanchor="top", y =1.7, bgcolor = 'rgba(0,0,0,0)'),
            template = 'simple_white',
            font = dict(size = 18)
            )

# create the figure
fig2 = go.Figure(data = data, layout = layout)

fig2.update_xaxes(tickangle=90)

fig2.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
       yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) )

fig2.write_image('{}/LHC_TEMP_&_Level_Sensor_Chart.png'.format(image_folder_name), engine="orca")
fig2.write_html('{}/LHC_TEMP_&_Level_Sensor_Chart.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# fig2.show()

# HYS 102 VS LVL 101 Chart

# temp_data = temp_data[temp_data['CURRENT SP'] >= 0]
temp_data = sys_data[(sys_data['MP STATE']=='STACK PWR') & (sys_data['MS STATE']=='ARMED')]
temp_data['CURRENT DENSITY'] = temp_data['STACK CURRENT'] / 702


HYS102 = go.Scattergl(
                       x = temp_data['Date'],
                       y = temp_data['HYS 102'],
                       mode = 'markers',
                       marker=dict(color='#ff7f0e'),
                       #marker=dict(size=3),
                       #mode = 'lines',
                       name = 'HYS 102 (% LEL)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )

LVL101  = go.Scattergl(
                        x = temp_data['Date'],
                        y = temp_data['LVL 101'],
                        mode = 'markers',
                        marker=dict(color='#1f77b4'),
                        #marker=dict(size=3),
                        #mode = 'lines',
                        name = 'LVL 101 (%)',
                        yaxis = 'y1',
                        #text = df.column (this is the info displayed on hover)
                        )
PRT401 = go.Scattergl(
                       x = temp_data['Date'],
                       y = temp_data['PRT 401'],
                       mode = 'markers',
                       marker=dict(color='#d62728'),
                       #mode = lines,
                       name = 'PRT 401 (bar)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
Current_Density = go.Scattergl(
                       x = temp_data['Date'],
                       y = temp_data['CURRENT DENSITY'],
                       mode = 'markers',
                       marker=dict(color='#2ca02c'),
#                        marker=dict(color='#e377c2'),
#                        mode = 'lines',
                       name = 'Current Density (A/c'+'m\u00b2)',
                       yaxis = 'y2'
                       #text = df.column (this is the info displayed on hover)
                       )

# update the data list
data = [Current_Density, PRT401,LVL101,HYS102]

# configure the layout
layout = go.Layout(
            title = 'LHC HYS 102 Vs. LVL 101',
            #to define axis, can either use go.layout.etc or dict(etc)
            xaxis = go.layout.XAxis(
#                                     title = 'Date',
                                    tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                    hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                    showgrid = False,
                                    #range=[100,steady_state['Duration(hr)'].max()]
                                    ),
            yaxis = dict(title=u'H\u2082 (% LEL) leak &'+'<br>'+' Water level (%)',
                                showgrid = False,
                                rangemode='tozero'),
            yaxis2= dict(title='Current Density'+'<br>'+'(A/c'+'m\u00b2)',
                                 overlaying='y',
                                 side = 'right',
                                 showgrid = False,
                                 rangemode='tozero'),
            legend= dict(xanchor="right",x=1.3,yanchor="top", y =1.6, bgcolor = 'rgba(0,0,0,0)'),
            template = 'simple_white',
            font = dict(size = 18)
            )

# create the figure
fig3 = go.Figure(data = data, layout = layout)

fig3.update_xaxes(tickangle=90)

fig3.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
       yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) )

fig3.write_image('{}/LHC_HYS_102_VS_LVL_101.png'.format(image_folder_name), engine="orca")
fig3.write_html('{}/LHC_HYS_102_VS_LVL_101.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# fig3.show()

# create LHC_HYS_102_&_LVL_101
HYS102 = go.Scattergl(
                       x = temp_data['Date'],
                       y = temp_data['HYS 102'],
                       mode = 'markers',
                       #marker=dict(size=3),
                       #mode = 'lines',
                       name = 'HYS 102 (% LEL)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )

LVL101  = go.Scattergl(
                        x = temp_data['Date'],
                        y = temp_data['LVL 101'],
                        mode = 'markers',
                        #marker=dict(size=3),
                        #mode = 'lines',
                        name = 'LVL 101 (%)',
                        yaxis = 'y2',
                        #text = df.column (this is the info displayed on hover)
                        )

# update the data list
data = [LVL101, HYS102]

# configure the layout
layout = go.Layout(
            title = 'LHC HYS 102 Vs. LVL 101',
            #to define axis, can either use go.layout.etc or dict(etc)
            xaxis = go.layout.XAxis(
#                                     title = 'Date',
                                    tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                    hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                    showgrid = False,
                                    #range=[100,steady_state['Duration(hr)'].max()]
                                    ),
            yaxis = dict(title=u'H\u2082 (% LEL) leak',
                                showgrid = False,
                                rangemode='tozero'),
            yaxis2= dict(title=' Water level (%)',
                                 overlaying='y',
                                 side = 'right',
                                 showgrid = False,
                                 rangemode='tozero'),
            legend= dict(xanchor="right",x=1.3,yanchor="top", y =1.4, bgcolor = 'rgba(0,0,0,0)'),
            template = 'simple_white',
            font = dict(size = 18)
            )

# create the figure
fig3 = go.Figure(data = data, layout = layout)

fig3.update_xaxes(tickangle=90)

fig3.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
       yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) )

fig3.write_image('{}/LHC_HYS_102_&_LVL_101.png'.format(image_folder_name), engine="orca")
fig3.write_html('{}/LHC_HYS_102_&_LVL_101.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# fig3.show()
Current_Density = go.Scattergl(
                       x = temp_data['Date'],
                       y = temp_data['CURRENT DENSITY'],
                       mode = 'markers',
                       marker=dict(color='#2ca02c'),
#                        mode = 'lines',
                       name = 'Current Density (A/c'+'m\u00b2)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
PRT401 = go.Scattergl(
                       x = temp_data['Date'],
                       y = temp_data['PRT 401'],
                       mode = 'markers',
                       #mode = lines,
                       marker=dict(color='#d62728'),
                       name = 'PRT 401 (bar)',
                       yaxis = 'y2'
                       #text = df.column (this is the info displayed on hover)
                       )
# update the data list
data = [Current_Density, PRT401]

# configure the layout
layout = go.Layout(
            title = 'PRT 401 and Current Density',
            #to define axis, can either use go.layout.etc or dict(etc)
            xaxis = go.layout.XAxis(
#                                     title = 'Date',
                                    tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                    hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                    showgrid = False,
                                    #range=[100,steady_state['Duration(hr)'].max()]
                                    ),
            yaxis= dict(title='Current Density'+'<br>'+'(A/c'+'m\u00b2)',
                                 showgrid = False,
                                 rangemode='tozero'),
            yaxis2 = dict(title= 'PRT 401 (bar)',
                                overlaying = 'y',
                                side = 'right',
                                showgrid = False,
                                rangemode='tozero'),
            legend= dict(xanchor="right",x=1.3,yanchor="top", y =1.4, bgcolor = 'rgba(0,0,0,0)'),
            template = 'simple_white',
            font = dict(size = 18)
            )

# create the figure
fig4 = go.Figure(data = data, layout = layout)

fig4.update_xaxes(tickangle=90)

fig4.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
       yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) )

fig4.write_image('{}/PRT_401_&_CD.png'.format(image_folder_name), engine="orca")
fig4.write_html('{}/PRT_401_&_CD.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# fig4.show()

# HYDROGEN & Water sensor charts
HYS101 = go.Scattergl(
                       x = sys_data['Date'],
                       y = sys_data['HYS 101'],
                       mode = 'markers',
                       #marker=dict(size=3),
                       #mode = 'lines',
                       name = 'HYS 101 (% LEL)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
HYS102 = go.Scattergl(
                       x = sys_data['Date'],
                       y = sys_data['HYS 102'],
                       mode = 'markers',
                       #marker=dict(size=3),
                       #mode = 'lines',
                       name = 'HYS 102 (% LEL)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
HYS401 = go.Scattergl(
                       x = sys_data['Date'],
                       y = sys_data['HYS 401'],
                       mode = 'markers',
                       #marker=dict(size=3),
                       #mode = lines,
                       name = 'HYS 401 (% LEL)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
HYS501  = go.Scattergl(
                        x = sys_data['Date'],
                        y = sys_data['HYS 501'],
                        mode = 'markers',
                        #marker=dict(size=3),
                        #mode = 'lines',
                        name = 'HYS 501 (% LEL)',
                        yaxis = 'y1',
                        #text = df.column (this is the info displayed on hover)
                        )
OXS101  = go.Scattergl(
                        x = sys_data['Date'],
                        y = sys_data['OXS 101'],
                        mode = 'markers',
                        #marker=dict(size=3),
                        #mode = 'lines',
                        name = 'OXS 101 (%)',
                        yaxis = 'y1',
                        #text = df.column (this is the info displayed on hover)
                        )
Current_Density = go.Scattergl(
                       x = sys_data['Date'],
                       y = sys_data['CURRENT DENSITY'],
                       mode = 'markers',
#                        mode = 'lines',
                       name = 'Current Density (A/c'+'m\u00b2)',
                       yaxis = 'y2'
                       #text = df.column (this is the info displayed on hover)
                       )

# update the data list
data = [HYS101, HYS102, HYS401, HYS501, OXS101, Current_Density]

# configure the layout
layout = go.Layout(
            title = 'LHC '+u'H\u2082 & O\u2082'+' leak',
            #to define axis, can either use go.layout.etc or dict(etc)
            xaxis = go.layout.XAxis(
#                                     title = 'Date',
                                    tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                    hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                    showgrid = False,
                                    #range=[100,steady_state['Duration(hr)'].max()]
                                    ),
            yaxis = dict(title=u'H\u2082 (% LEL) & O\u2082 (%) leak',
                                showgrid = False,
                                rangemode='tozero'),
            yaxis2= dict(title='Current Density'+'<br>'+'(A/c'+'m\u00b2)',
                                 overlaying='y',
                                 side = 'right',
                                 showgrid = False,
                                 range = [0,6],
                                 rangemode='tozero'),
            legend= dict(xanchor="right",x=1.3,yanchor="top", y =2.0, bgcolor = 'rgba(0,0,0,0)'),
            template = 'simple_white',
            font = dict(size = 18)
            )

# create the figure
fig3 = go.Figure(data = data, layout = layout)

fig3.update_xaxes(tickangle=90)

fig3.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
       yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) )

fig3.write_image('{}/LHC_H2_&_O2_leak.png'.format(image_folder_name), engine="orca")
fig3.write_html('{}/LHC_H2_&_O2_leak.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# fig3.show()

# LWC - PRT 704 and PMP state chart 

sys_data['PMP_ON_OFF'] = np.where(sys_data['PMP 102'] == 'ON',1,0)

PRT704 = go.Scattergl(
                       x = sys_data['Date'],
                       y = sys_data['PRT 704'],
                       mode = 'markers',
                       #marker=dict(size=3),
#                        mode = 'lines',
                       name = 'PRT 704 (bar)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
PMP102  = go.Scattergl(
                        x = sys_data['Date'],
                        y = sys_data['PMP_ON_OFF'],
                        mode = 'markers',
                        #marker=dict(size=3),
                        #mode = 'lines',
                        name = 'PMP State',
                        yaxis = 'y2',
                        #text = df.column (this is the info displayed on hover)
                        )

# update the data list
data = [PRT704, PMP102]

# configure the layout
layout = go.Layout(
            title = 'LWC Pressure (PRT 704)'+'<br>'+'and PMP State',
            #to define axis, can either use go.layout.etc or dict(etc)
            xaxis = go.layout.XAxis(
#                                     title = 'Date',
                                    tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                    hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                    showgrid = False,
                                    #range=[100,steady_state['Duration(hr)'].max()]
                                    ),    
            yaxis = dict(title='Pressure (bar)',
                                showgrid = False,
                                rangemode='tozero'),
            yaxis2= dict(title='PMP STATE',
                                 overlaying='y',
                                 side = 'right',
                                 tickvals=[0,1],
                                 showgrid = False,
#                                  rangemode='tozero',
                                 range = [-0.1,1.1]
#                                  range=list(range(0,2,1))
                        ),
            legend = dict(xanchor="right",x=1.4,yanchor="top", y =1.6, bgcolor = 'rgba(0,0,0,0)'),
            template = 'simple_white',
            font = dict(size = 18)
            )

# create the figure
fig10 = go.Figure(data = data, layout = layout)

# ,row=1, col=1)

fig10.update_xaxes(tickangle=90)

fig10.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
                  yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
                  yaxis2 = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) )

fig10.write_image('{}/LWC_Pressure(PRT 704)_and_PMP_State.png'.format(image_folder_name), engine="orca")
fig10.write_html('{}/LWC_Pressure(PRT 704)_and_PMP_State.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# fig10.show()

# LWC - PRT 705 and PMP State chart

sys_data['PMP_ON_OFF'] = np.where(sys_data['PMP 102'] == 'ON',1,0)


PRT705 = go.Scattergl(
                       x = sys_data['Date'],
                       y = sys_data['PRT 705'],
                       mode = 'markers',
                       #marker=dict(size=3),
#                        mode = 'lines',
                       name = 'PRT 705 (bar)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
# PRT706  = go.Scattergl(
#                         x = sys_data['Date'],
#                         y = sys_data['PRT 706'],
#                         mode = 'markers',
#                         #marker=dict(size=3),
#                         #mode = 'lines',
#                         name = 'PRT 706 (bar)',
#                         yaxis = 'y1',
#                         #text = df.column (this is the info displayed on hover)
#                         )
PMP102  = go.Scattergl(
                        x = sys_data['Date'],
                        y = sys_data['PMP_ON_OFF'],
                        mode = 'markers',
                        #marker=dict(size=3),
                        #mode = 'lines',
                        name = 'PMP State',
                        yaxis = 'y2',
                        #text = df.column (this is the info displayed on hover)
                        )

# update the data list
data = [PRT705, PMP102]

# configure the layout
layout = go.Layout(
            title = 'LWC Pressure (PRT 705)'+'<br>'+'and PMP State',
            #to define axis, can either use go.layout.etc or dict(etc)
            xaxis = go.layout.XAxis(
#                                     title = 'Date',
                                    tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                    hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                    showgrid = False,
                                    #range=[100,steady_state['Duration(hr)'].max()]
                                    ),    
            yaxis = dict(title='Pressure (bar)',
                                showgrid = False,
                                rangemode='tozero'),
            yaxis2= dict(title='PMP STATE',
                                 overlaying='y',
                                 side = 'right',
                                 tickvals=[0,1],
                                 showgrid = False,
                                 range = [-0.1,1.1]
#                                  rangemode='tozero',
#                                  range=list(range(0,2,1))
                        ),
            legend = dict(xanchor="right",x=1.4,yanchor="top", y =1.6, bgcolor = 'rgba(0,0,0,0)'),
            template = 'simple_white',
            font = dict(size = 18)
            )

# create the figure
fig10 = go.Figure(data = data, layout = layout)

# ,row=1, col=1)

fig10.update_xaxes(tickangle=90)

fig10.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
                  yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
                  yaxis2 = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) )

fig10.write_image('{}/LWC_Pressure(PRT 705)_and_PMP_State.png'.format(image_folder_name), engine="orca")
fig10.write_html('{}/LWC_Pressure(PRT 705)_and_PMP_State.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# fig10.show()

# LWC COS701 , COS702 Conductivity 
COS701 = go.Scattergl(
                       x = sys_data['Date'],
                       y = sys_data['COS 701'],
                       mode = 'markers',
                       #marker=dict(size=3),
                       #mode = 'lines',
                       name = 'COS 701 (µS/cm)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
COS702 = go.Scattergl(
                       x = sys_data['Date'],
                       y = sys_data['COS 702'],
                       mode = 'markers',
                       #marker=dict(size=3),
                       #mode = lines,
                       name = 'COS 702 (µS/cm)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
# update the data list
data = [COS701, COS702]

# configure the layout
layout = go.Layout(
            title = 'LWC Conductivity',
            #to define axis, can either use go.layout.etc or dict(etc)
            xaxis = go.layout.XAxis(
#                                     title = 'Date',
                                    tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                    hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                    showgrid = False,
                                    #range=[100,steady_state['Duration(hr)'].max()]
                                    ),    
            yaxis = dict(title='Conductivity (µS/cm)',
                                showgrid = False,
                                rangemode='tozero'),
            yaxis2= dict(title='Current Density'+'<br>' +'(A/c'+'m\u00b2)',
                                 overlaying='y',
                                 side = 'right',
                                 range = [0,2],
                                 showgrid = False,
                                 rangemode='tozero'),
#                                  range=[0,600]),
            legend = dict(xanchor="right",x=1.4,yanchor="top", y =1.3, bgcolor = 'rgba(0,0,0,0)'),
            template = 'simple_white',
            font = dict(size = 18)
            )

# create the figure
fig11 = go.Figure(data = data, layout = layout)

fig11.update_xaxes(tickangle=90)

fig11.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
                  yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
#                   yaxis2 = dict(titlefont = dict(size = 15), tickfont = dict(size = 15)) 
                 )


fig11.write_image('{}/LWC_Conductivity.png'.format(image_folder_name), engine="orca")
fig11.write_html('{}/LWC_Conductivity.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# fig11.show()
print("All charts are successfully exported")

# ##PSU Voltage and Current charts

# ##set the lower and upper limit for PSU cols
# columns_list_V = ['PSU1 V', 'PSU2 V', 'PSU3 V', 'PSU4 V', 'PSU5 V', 'PSU6 V', 'PSU7 V', 'PSU8 V', 'PSU9 V', 'PSU10 V', 'PSU11 V', 
#                'PSU12 V', 'PSU13 V', 'PSU14 V', 'PSU15 V', 'PSU16 V', 'PSU17 V', 'PSU18 V', 'PSU19 V', 'PSU20 V', 'PSU21 V',
#                'PSU22 V', 'PSU23 V', 'PSU24 V', 'PSU25 V', 'PSU26 V']

# columns_list_I = ['PSU1 I', 'PSU2 I', 'PSU3 I', 'PSU4 I', 'PSU5 I', 'PSU6 I', 'PSU7 I', 'PSU8 I', 'PSU9 I', 'PSU10 I', 'PSU11 I', 
#                'PSU12 I', 'PSU13 I', 'PSU14 I', 'PSU15 I', 'PSU16 I', 'PSU17 I', 'PSU18 I', 'PSU19 I', 'PSU20 I', 'PSU21 I',
#                'PSU22 I', 'PSU23 I', 'PSU24 I', 'PSU25 I', 'PSU26 I']

# condition = (sys_data[columns_list_V ] >= 0) & (sys_data[columns_list_V ] <= 400)
# sys_data = sys_data[condition.all(axis=1)]
    
# condition = (sys_data[columns_list_I] >= 0) & (sys_data[columns_list_I] <= 41)
# sys_data = sys_data[condition.all(axis=1)]

# sys_data['Sum_PSU_V'] = sys_data[columns_list_V].sum(axis=1)
# sys_data['Sum_PSU_I'] = sys_data[columns_list_I].sum(axis=1)

# sys_data['Avg_PSU_V'] = sys_data[columns_list_V].sum(axis=1)/26
# sys_data['Avg_PSU_I'] = sys_data[columns_list_I].sum(axis=1)/26

# sys_data['max_PSU_V'] = sys_data[columns_list_V].max(axis=1)
# sys_data['max_PSU_I'] = sys_data[columns_list_I].max(axis=1)

# sys_data['min_PSU_V'] = sys_data[columns_list_V].min(axis=1)
# sys_data['min_PSU_I'] = sys_data[columns_list_I].min(axis=1)

# Avg_PSU_V = go.Scattergl(
#                        x = sys_data['Date'],
#                        y = sys_data['Avg_PSU_V'],
#                        mode = 'markers',
#                        #marker=dict(size=3),
# #                        mode = 'lines',
#                        name = 'Avg PSU Voltage (V)',
#                        yaxis = 'y1'
#                        #text = df.column (this is the info displayed on hover)
#                        )
# max_PSU_V = go.Scattergl(
#                        x = sys_data['Date'],
#                        y = sys_data['max_PSU_V'],
#                        mode = 'markers',
#                        #marker=dict(size=3),
#                        #mode = lines,
#                        name = 'Max PSU Voltage (V)',
#                        yaxis = 'y1'
#                        #text = df.column (this is the info displayed on hover)
#                        )
# min_PSU_V = go.Scattergl(
#                        x = sys_data['Date'],
#                        y = sys_data['min_PSU_V'],
#                        mode = 'markers',
#                        #marker=dict(size=3),
#                        #mode = lines,
#                        name = 'Min PSU Voltage (V)',
#                        yaxis = 'y1'
#                        #text = df.column (this is the info displayed on hover)
#                        )

# # update the data list
# data = [Avg_PSU_V, max_PSU_V, min_PSU_V]

# # configure the layout
# layout = go.Layout(
#             title = 'LPC PSU Voltage',
#             #to define axis, can either use go.layout.etc or dict(etc)
#             xaxis = go.layout.XAxis(
# #                                     title = 'Date',
#                                     tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
#                                     hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
#                                     showgrid = False,
#                                     #range=[100,steady_state['Duration(hr)'].max()]
#                                     ),    
#             yaxis = dict(title='Voltage (V)',
#                                 showgrid = False,
#                                 rangemode='tozero'),
# #             yaxis2= dict(title='PMP STATE',
# #                                  overlaying='y',
# #                                  side = 'right',
# #                                  showgrid = False,
# #                                  rangemode='tozero'),
# #                                  range=[0,600]),
#             legend = dict(xanchor="right",x=1.3,yanchor="top", y =1.5, bgcolor = 'rgba(0,0,0,0)'),
#             template = 'simple_white',
#             font = dict(size = 18)
#             )

# # create the figure
# fig12 = go.Figure(data = data, layout = layout)

# fig12.update_xaxes(tickangle=90)

# fig12.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
#                   yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
# #                   yaxis2 = dict(titlefont = dict(size = 15), tickfont = dict(size = 15)) 
#                  )

# fig12.write_image('{}/LPC_PSU_Voltage.png'.format(image_folder_name), engine="orca")
# fig12.write_html('{}/LPC_PSU_Voltage.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# # fig12.show()

# # PSU_Current Chart(Avg, Min, Max)

# Avg_PSU_I = go.Scattergl(
#                        x = sys_data['Date'],
#                        y = sys_data['Avg_PSU_I'],
#                        mode = 'markers',
#                        #marker=dict(size=3),
# #                        mode = 'lines',
#                        name = 'Avg PSU Current (A)',
#                        yaxis = 'y1'
#                        #text = df.column (this is the info displayed on hover)
#                        )
# max_PSU_I = go.Scattergl(
#                        x = sys_data['Date'],
#                        y = sys_data['max_PSU_I'],
#                        mode = 'markers',
#                        #marker=dict(size=3),
# #                        mode = 'lines',
#                        name = 'Max PSU Current (A)',
#                        yaxis = 'y1'
#                        #text = df.column (this is the info displayed on hover)
#                        )
# min_PSU_I = go.Scattergl(
#                        x = sys_data['Date'],
#                        y = sys_data['min_PSU_I'],
#                        mode = 'markers',
#                        #marker=dict(size=3),
# #                        mode = 'lines',
#                        name = 'Min PSU Current (A)',
#                        yaxis = 'y1'
#                        #text = df.column (this is the info displayed on hover)
#                        )

# # update the data list
# data = [Avg_PSU_I, max_PSU_I, min_PSU_I]

# # configure the layout
# layout = go.Layout(
#             title = 'LPC PSU Current',
#             #to define axis, can either use go.layout.etc or dict(etc)
#             xaxis = go.layout.XAxis(
# #                                      title = 'Date',
#                                     tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
#                                     hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
#                                     showgrid = False,
#                                     #range=[100,steady_state['Duration(hr)'].max()]
#                                     ),    
#             yaxis = dict(title='Current (A)',
#                                 showgrid = False,
#                                 rangemode='tozero'),
# #             yaxis2= dict(title='PMP STATE',
# #                                  overlaying='y',
# #                                  side = 'right',
# #                                  showgrid = False,
# #                                  rangemode='tozero'),
# #                                  range=[0,600]),
#             legend = dict(xanchor="right",x=1.3,yanchor="top", y =1.5, bgcolor = 'rgba(0,0,0,0)'),
#             template = 'simple_white',
#             font = dict(size = 18)
#             )

# # create the figure
# fig13 = go.Figure(data = data, layout = layout)

# fig13.update_xaxes(tickangle=90)

# fig13.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
#                   yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
# #                   yaxis2 = dict(titlefont = dict(size = 15), tickfont = dict(size = 15)) 
#                  )
# fig13.write_image('{}/LPC_PSU_Current.png'.format(image_folder_name), engine="orca")
# fig13.write_html('{}/LPC_PSU_Current.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# # fig13.show()

# ############################################## Plot Cell C13-2 and C15-15 ###Bad Cells  ###############################3
# Current_Density = go.Scattergl(
#                        x = temp_data['Date'],
#                        y = temp_data['CURRENT DENSITY'],
#                        mode = 'markers',
#                        marker=dict(color='#2ca02c'),
# #                        mode = 'lines',
#                        name = 'Current Density (A/c'+'m\u00b2)',
#                        yaxis = 'y1'
#                        #text = df.column (this is the info displayed on hover)
#                        )
# C13_2 = go.Scattergl(
#                        x = temp_data['Date'],
#                        y = temp_data['C13-2'],
#                        mode = 'markers',
#                        #mode = lines,
#                        marker=dict(color='#d62728'),
#                        name = 'C13-2 (V)',
#                        yaxis = 'y2'
#                        #text = df.column (this is the info displayed on hover)
#                        )
# # update the data list
# data = [C13_2,Current_Density]

# # configure the layout
# layout = go.Layout(
#             title = 'C13-2 and Current Density',
#             #to define axis, can either use go.layout.etc or dict(etc)
#             xaxis = go.layout.XAxis(
# #                                     title = 'Date',
#                                     tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
#                                     hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
#                                     showgrid = False,
#                                     #range=[100,steady_state['Duration(hr)'].max()]
#                                     ),
#             yaxis2 = dict(title= 'Voltage (V)',
#                                 range = [1.3,2.3],
#                                 side = 'left',
#                                 showgrid = False,
#                                 rangemode='tozero'),
#             yaxis= dict(title='Current Density'+'<br>'+'(A/c'+'m\u00b2)',
#                                  overlaying = 'y',
#                                  side = 'right',
#                                  showgrid = False,
#                                  rangemode='tozero'),
            
#             legend= dict(xanchor="right",x=1.3,yanchor="top", y =1.4, bgcolor = 'rgba(0,0,0,0)'),
#             template = 'simple_white',
#             font = dict(size = 18)
#             )

# # create the figure
# fig4 = go.Figure(data = data, layout = layout)

# fig4.update_xaxes(tickangle=90)

# fig4.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
#        yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) )

# fig4.write_image('{}/C13-2_&_CD.png'.format(image_folder_name), engine="orca")
# fig4.write_html('{}/C13-2_&_CD.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# # fig4.show()




# Current_Density = go.Scattergl(
#                        x = temp_data['Date'],
#                        y = temp_data['CURRENT DENSITY'],
#                        mode = 'markers',
#                        marker=dict(color='#2ca02c'),
# #                        mode = 'lines',
#                        name = 'Current Density (A/c'+'m\u00b2)',
#                        yaxis = 'y1'
#                        #text = df.column (this is the info displayed on hover)
#                        )
# C13_15 = go.Scattergl(
#                        x = temp_data['Date'],
#                        y = temp_data['C15-15'],
#                        mode = 'markers',
#                        #mode = lines,
#                        marker=dict(color='#d62728'),
#                        name = 'C15-15 (V)',
#                        yaxis = 'y2'
#                        #text = df.column (this is the info displayed on hover)
#                        )
# # update the data list
# data = [C13_15,Current_Density]

# # configure the layout
# layout = go.Layout(
#             title = 'C15-15 and Current Density',
#             #to define axis, can either use go.layout.etc or dict(etc)
#             xaxis = go.layout.XAxis(
# #                                     title = 'Date',
#                                     tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
#                                     hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
#                                     showgrid = False,
#                                     #range=[100,steady_state['Duration(hr)'].max()]
#                                     ),
#             yaxis2 = dict(title= 'Voltage (V)',
#                                 range = [1.3, 2.3],
#                                 side = 'left',
#                                 showgrid = False,
#                                 rangemode='tozero'),
#             yaxis= dict(title='Current Density'+'<br>'+'(A/c'+'m\u00b2)',
#                                  overlaying = 'y',
#                                  side = 'right',
#                                  showgrid = False,
#                                  rangemode='tozero'),
            
#             legend= dict(xanchor="right",x=1.3,yanchor="top", y =1.4, bgcolor = 'rgba(0,0,0,0)'),
#             template = 'simple_white',
#             font = dict(size = 18)
#             )

# # create the figure
# fig4 = go.Figure(data = data, layout = layout)

# fig4.update_xaxes(tickangle=90)

# fig4.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
#        yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) )

# fig4.write_image('{}/C15-15_&_CD.png'.format(image_folder_name), engine="orca")
# fig4.write_html('{}/C15-15_&_CD.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# # fig4.show()
# print("All charts are successfully exported")

All charts are successfully exported


In [14]:
print('done')

done


In [15]:
# check if all Vcell cols are blank for a row then, drop that row
Stack_voltage_list = [
    'C1-1', 'C1-2', 'C1-3', 'C1-4', 'C1-5', 'C1-6', 'C1-7', 'C1-8', 'C1-9', 'C1-10', 'C1-11', 'C1-12', 'C1-13', 'C1-14', 'C1-15', 'C1-16', 'C2-1', 'C2-2', 'C2-3', 'C2-4', 'C2-5', 'C2-6', 'C2-7', 'C2-8', 'C2-9', 'C2-10', 'C2-11', 'C2-12', 'C2-13', 'C2-14', 'C2-15', 'C2-16', 'C3-1', 'C3-2', 'C3-3', 'C3-4', 'C3-5', 'C3-6', 'C3-7', 'C3-8', 'C3-9', 'C3-10', 'C3-11', 'C3-12', 'C3-13', 'C3-14', 'C3-15', 'C3-16', 'C4-1', 'C4-2', 'C4-3', 'C4-4', 'C4-5', 'C4-6', 'C4-7', 'C4-8'
#                       , 'C4-9', 'C4-10', 'C4-11', 'C4-12', 'C4-13', 'C4-14', 'C4-15', 'C4-16',
    ,'C5-1', 'C5-2', 'C5-3', 'C5-4', 'C5-5', 'C5-6', 'C5-7', 'C5-8', 'C5-9', 'C5-10', 'C5-11', 'C5-12', 'C5-13', 'C5-14', 'C5-15', 'C5-16', 'C6-1', 'C6-2', 'C6-3', 'C6-4', 'C6-5', 'C6-6', 'C6-7', 'C6-8', 'C6-9', 'C6-10', 'C6-11', 'C6-12', 'C6-13', 'C6-14', 'C6-15', 'C6-16', 'C7-1', 'C7-2', 'C7-3', 'C7-4', 'C7-5', 'C7-6', 'C7-7', 'C7-8', 'C7-9', 'C7-10', 'C7-11', 'C7-12', 'C7-13', 'C7-14', 'C7-15', 'C7-16', 'C8-1', 'C8-2', 'C8-3', 'C8-4', 'C8-5', 'C8-6', 'C8-7', 'C8-8'
#                       , 'C8-9', 'C8-10', 'C8-11', 'C8-12', 'C8-13', 'C8-14', 'C8-15', 'C8-16',
    ,'C9-1', 'C9-2', 'C9-3', 'C9-4', 'C9-5', 'C9-6', 'C9-7', 'C9-8', 'C9-9', 'C9-10', 'C9-11', 'C9-12', 'C9-13', 'C9-14', 'C9-15', 'C9-16', 'C10-1', 'C10-2', 'C10-3', 'C10-4', 'C10-5', 'C10-6', 'C10-7', 'C10-8', 'C10-9', 'C10-10', 'C10-11', 'C10-12', 'C10-13', 'C10-14', 'C10-15', 'C10-16', 'C11-1', 'C11-2', 'C11-3', 'C11-4', 'C11-5', 'C11-6', 'C11-7', 'C11-8', 'C11-9', 'C11-10', 'C11-11', 'C11-12', 'C11-13', 'C11-14', 'C11-15', 'C11-16','C12-1', 'C12-2', 'C12-3', 'C12-4', 'C12-5', 'C12-6', 'C12-7', 'C12-8'
#                       , 'C12-9', 'C12-10', 'C12-11', 'C12-12', 'C12-13', 'C12-14', 'C12-15', 'C12-16',
    ,'C13-1', 'C13-2', 'C13-3', 'C13-4', 'C13-5', 'C13-6', 'C13-7', 'C13-8', 'C13-9', 'C13-10', 'C13-11', 'C13-12', 'C13-13', 'C13-14', 'C13-15', 'C13-16', 'C14-1', 'C14-2', 'C14-3', 'C14-4', 'C14-5', 'C14-6', 'C14-7', 'C14-8', 'C14-9', 'C14-10', 'C14-11', 'C14-12', 'C14-13', 'C14-14', 'C14-15', 'C14-16', 'C15-1', 'C15-2', 'C15-3', 'C15-4', 'C15-5', 'C15-6', 'C15-7', 'C15-8', 'C15-9', 'C15-10', 'C15-11', 'C15-12', 'C15-13', 'C15-14', 'C15-15', 'C15-16', 'C16-1', 'C16-2', 'C16-3', 'C16-4', 'C16-5', 'C16-6', 'C16-7', 'C16-8'
#              , 'C16-9', 'C16-10', 'C16-11', 'C16-12', 'C16-13', 'C16-14', 'C16-15', 'C16-16'
]


In [16]:
# Drop rows where all the specified columns are blank
Vcell_data.dropna(subset=Stack_voltage_list, how='all', inplace=True)
# make a copy of the relevant df
temp = Vcell_data.loc[:,['Date','CURRENT DENSITY']].copy()
# identify pol curve end
# this is defined here as where the difference in rolling mean is < -.05 and current density is > than 0.5
end_condition1 = (temp['CURRENT DENSITY'].rolling(5, min_periods=1, center=True).mean() - temp['CURRENT DENSITY'].shift().rolling(5, min_periods=1, center=True).mean()) < -0.01
# print(end_condition1.value_counts())
end_condition2 = (temp['CURRENT DENSITY']) > 0.1
# print(end_condition2.value_counts())
end_condition = end_condition1.shift(-1, fill_value=False) & end_condition2
endpoints = temp.loc[end_condition, :].copy()
endpoints['type'] = 1
# print(endpoints)
# idnetify pol curve starts
# this is defined here as where the current density increases by > 0.001 and is less than 0.02
start_condition1 = (temp['CURRENT DENSITY'] - temp['CURRENT DENSITY'].shift()) > 0.001
start_condition2 = (temp['CURRENT DENSITY']) < 0.02
start_condition = start_condition1.shift(-1, fill_value=False) & start_condition2
startpoints = temp.loc[start_condition, :].copy()
startpoints['type'] = 0
# print(startpoints)
# put them together
pol_times = pd.concat([startpoints, endpoints])
pol_times.sort_values(by=['Date'], inplace=True)
# get only the start and endpoints that border each other
change_condition = (pol_times['type'] - pol_times['type'].shift()) == 1
pol_times = pol_times.loc[(change_condition | change_condition.shift(-1, fill_value=False)), :]
# print(pol_times)
# check if monotonically increasing for each interval
t_start = []
t_end = []
# for each interval
for i in range(len(pol_times.loc[pol_times['type'] == 0, 'Date'])):
    # get the start and end time of the overall interval
    start = pol_times.iloc[i*2]['Date']
    end = pol_times.iloc[(i*2)+1]['Date']
    # sample the data at nominal frequency between start and end
    sample = temp[(temp['Date'] >= start) & (temp['Date'] <= end)]
    # if there is less than 18 points, then dont use this interval
#     print(len(sample))
    if len(sample) < 18:
        continue
    else:
        # make the rolling window size 10% of interval length
        win = int(len(sample)/10)
        # sample the data based on this window size and take the mean
        sample_sparse = sample['CURRENT DENSITY'].rolling(win, min_periods=1, center=True, step = win).mean() 
        # while the data is monotonically increasing (by more than 0.01)
        idx = 1
        while (sample_sparse.values[idx] - sample_sparse.values[idx-1]) >-0.1:
            # update the counter
            idx += 1
            if idx >= len(sample_sparse):
                break
        # update the end point based on the counter if the counter did not reach the full len
        if idx < len(sample_sparse):
            end = sample.loc[sample_sparse.index[idx-1], 'Date']
            # resample the data at nominal rate using the updated endpoint
            sample = temp[(temp['Date'] >= start) & (temp['Date'] <= end)]
        # if the resulting pol interval has at least 18 points, then update the relevant lists
#         print(len(sample))
        if len(sample) > 18:
            t_start.append(start)
            t_end.append(end)
# print(list(t_start))
# print(list(t_end)) 

##Plotting cd & identified pol curves plot and polcurve plots
def pol_curve_func():
    mask = []
    for i,j in zip(t_start,t_end):
        temp_t = Vcell_data.loc[(Vcell_data['Date'] >= i ) & (Vcell_data['Date'] <= j), ['Date','CURRENT DENSITY']]
        mask.append(temp_t)

    stacked_df = pd.DataFrame(columns=temp_t.columns)
    for temp_t in mask:
        stacked_df = stacked_df.append(temp_t, ignore_index=True)
    stacked_df.rename(columns={'CURRENT DENSITY':'pol_curves'}, inplace = True) 


    Current_Density = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['CURRENT DENSITY'],
                           mode = 'markers',
                           marker=dict(color='#2ca02c'),
    #                        mode = 'lines',
                           name = 'Current Density (A/c'+'m\u00b2)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    pol_curves = go.Scattergl(
                           x = stacked_df['Date'],
                           y = stacked_df['pol_curves'],
                           mode = 'markers',
    #                        marker=dict(color='#2ca02c'),
    #                        mode = 'lines',
                           name = 'pol_curves',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )

    # update the data list
    data = [Current_Density,pol_curves]

    # configure the layout
    layout = go.Layout(
                title = 'Current Density',
                #to define axis, can either use go.layout.etc or dict(etc)
                xaxis = go.layout.XAxis(
    #                                     title = 'Date',
                                        tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                        hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                        showgrid = False,
                                        #range=[100,steady_state['Duration(hr)'].max()]
                                        ),    
                yaxis= dict(title='Current Density'+'<br>'+'(A/c'+'m\u00b2)',
    #                                  overlaying='y',
                                     side = 'left',
                                     showgrid = False,
    #                                  range = [0,6],
                                     rangemode='tozero'),
                legend= dict(xanchor="right",x=1.5,yanchor="top", y =2, bgcolor = 'rgba(0,0,0,0)'),
                template = 'simple_white',
                font = dict(size = 18)
                )


    # create the figure
    fig0 = go.Figure(data = data, layout = layout)
    fig0.update_xaxes(tickangle=90)

    fig0.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
           yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
           yaxis2 = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) 
                     )

    fig0.write_image('{}/Cd_&_polcurves.png'.format(image_folder_name), engine="orca")
    fig0.write_html('{}/Cd_&_polcurves.html'.format(image_folder_name), include_plotlyjs=('cdn'))
    # fig0.show()
    # specify the run duration completed at the polcurve start
    T1 = t_start
    T2 = t_end
    print(T1)
    print(T2)
    N=[]
    for i in T1:
    #     print(Vcell_data[Vcell_data['Date'] == i][['Date','DURATION_HR']])
        d = str(round(Vcell_data[Vcell_data['Date'] == i]['DURATION_HR'].iloc[0].item(),1))+" hrs"
        N.append(d)
    print(N)
    
    ####################################### POL CURVE CREATION #################################################################
    Stack_voltage_list = [
        'C1-1', 'C1-2', 'C1-3', 'C1-4', 'C1-5', 'C1-6', 'C1-7', 'C1-8', 'C1-9', 'C1-10', 'C1-11', 'C1-12', 'C1-13', 'C1-14', 'C1-15', 'C1-16', 'C2-1', 'C2-2', 'C2-3', 'C2-4', 'C2-5', 'C2-6', 'C2-7', 'C2-8', 'C2-9', 'C2-10', 'C2-11', 'C2-12', 'C2-13', 'C2-14', 'C2-15', 'C2-16', 'C3-1', 'C3-2', 'C3-3', 'C3-4', 'C3-5', 'C3-6', 'C3-7', 'C3-8', 'C3-9', 'C3-10', 'C3-11', 'C3-12', 'C3-13', 'C3-14', 'C3-15', 'C3-16', 'C4-1', 'C4-2', 'C4-3', 'C4-4', 'C4-5', 'C4-6', 'C4-7', 'C4-8'
    #                       , 'C4-9', 'C4-10', 'C4-11', 'C4-12', 'C4-13', 'C4-14', 'C4-15', 'C4-16',
        ,'C5-1', 'C5-2', 'C5-3', 'C5-4', 'C5-5', 'C5-6', 'C5-7', 'C5-8', 'C5-9', 'C5-10', 'C5-11', 'C5-12', 'C5-13', 'C5-14', 'C5-15', 'C5-16', 'C6-1', 'C6-2', 'C6-3', 'C6-4', 'C6-5', 'C6-6', 'C6-7', 'C6-8', 'C6-9', 'C6-10', 'C6-11', 'C6-12', 'C6-13', 'C6-14', 'C6-15', 'C6-16', 'C7-1', 'C7-2', 'C7-3', 'C7-4', 'C7-5', 'C7-6', 'C7-7', 'C7-8', 'C7-9', 'C7-10', 'C7-11', 'C7-12', 'C7-13', 'C7-14', 'C7-15', 'C7-16', 'C8-1', 'C8-2', 'C8-3', 'C8-4', 'C8-5', 'C8-6', 'C8-7', 'C8-8'
    #                       , 'C8-9', 'C8-10', 'C8-11', 'C8-12', 'C8-13', 'C8-14', 'C8-15', 'C8-16',
        ,'C9-1', 'C9-2', 'C9-3', 'C9-4', 'C9-5', 'C9-6', 'C9-7', 'C9-8', 'C9-9', 'C9-10', 'C9-11', 'C9-12', 'C9-13', 'C9-14', 'C9-15', 'C9-16', 'C10-1', 'C10-2', 'C10-3', 'C10-4', 'C10-5', 'C10-6', 'C10-7', 'C10-8', 'C10-9', 'C10-10', 'C10-11', 'C10-12', 'C10-13', 'C10-14', 'C10-15', 'C10-16', 'C11-1', 'C11-2', 'C11-3', 'C11-4', 'C11-5', 'C11-6', 'C11-7', 'C11-8', 'C11-9', 'C11-10', 'C11-11', 'C11-12', 'C11-13', 'C11-14', 'C11-15', 'C11-16','C12-1', 'C12-2', 'C12-3', 'C12-4', 'C12-5', 'C12-6', 'C12-7', 'C12-8'
    #                       , 'C12-9', 'C12-10', 'C12-11', 'C12-12', 'C12-13', 'C12-14', 'C12-15', 'C12-16',
        ,'C13-1', 'C13-2', 'C13-3', 'C13-4', 'C13-5', 'C13-6', 'C13-7', 'C13-8', 'C13-9', 'C13-10', 'C13-11', 'C13-12', 'C13-13', 'C13-14', 'C13-15', 'C13-16', 'C14-1', 'C14-2', 'C14-3', 'C14-4', 'C14-5', 'C14-6', 'C14-7', 'C14-8', 'C14-9', 'C14-10', 'C14-11', 'C14-12', 'C14-13', 'C14-14', 'C14-15', 'C14-16', 'C15-1', 'C15-2', 'C15-3', 'C15-4', 'C15-5', 'C15-6', 'C15-7', 'C15-8', 'C15-9', 'C15-10', 'C15-11', 'C15-12', 'C15-13', 'C15-14', 'C15-15', 'C15-16', 'C16-1', 'C16-2', 'C16-3', 'C16-4', 'C16-5', 'C16-6', 'C16-7', 'C16-8'
    #              , 'C16-9', 'C16-10', 'C16-11', 'C16-12', 'C16-13', 'C16-14', 'C16-15', 'C16-16'
    ]

    n=N
    t1=T1
    t2=T2

    #creating lists to later make Min/Max Cell table from these
    N_temp=N.copy()
    T1_temp=T1.copy()
    T2_temp=T2.copy()

    fig = go.Figure()

    for n,t1,t2 in zip(n,t1,t2):

        pol_duration_df = Vcell_data[(Vcell_data.Date >= t1) & (Vcell_data.Date <= t2)]
        poldata_list = Stack_voltage_list.copy()
        poldata_list.extend(['Date','CURRENT SP','STACK CURRENT','CURRENT DENSITY'])
        pol_data = pol_duration_df[poldata_list]

        ## choose indexes that have only one row for a current SP  
        indexes = pol_data[pol_data.groupby("CURRENT SP")["CURRENT SP"].transform('size') == 1].index
        ## and then drop those rows
        pol_data.drop(indexes, axis=0, inplace=True)
        df_polcurve1 = pd.DataFrame(columns = list(pol_data.columns))
        list_curr_sp = pol_data['CURRENT SP'].unique()
        for i in list_curr_sp:
            df_temp = pol_data[pol_data['CURRENT SP']==i]
            df_polcurve1 = df_polcurve1.append(df_temp.iloc[-2])    
        df_polcurve1 = df_polcurve1.reset_index(drop=True)    
        ############################# added logic to remove decreasing CD
        ##create a list out of sc column
        polcurve1_SC = df_polcurve1['STACK CURRENT']
        polcurve1_SC = list(polcurve1_SC)
        # check if next element of list is decreasing , remove it
        index = 0
        while index < (len(polcurve1_SC)-1):
            diff = polcurve1_SC[index + 1] - polcurve1_SC[index]
            if diff < 3:
                polcurve1_SC.pop(index + 1)  # Remove the next element
            else:
                index += 1  # Move to the next element if no removal was made

        ## writing a condition that if unique data pnts in the polcurve data is less than 3, then drop that polcurvre and continue to next polcurve        
        if len(polcurve1_SC)<3:
            N_temp.remove(n)
            T1_temp.remove(t1)
            T2_temp.remove(t2)
            continue
        # else go for making the polcurve further    
        else:                
            # create a new dataframe out of polcurve1_SC list and join with the original df to pick required data rows.
            df_polcurve2 = pd.DataFrame()
            df_polcurve2['STACK CURRENT'] =  polcurve1_SC
            ## joining two dfs
            df_polcurve1 = df_polcurve2.merge(df_polcurve1, on='STACK CURRENT', how='left')
            ###############################
        #     print(df_polcurve1)
    #         df_polcurve1['Avg_V'] = df_polcurve1[Stack_voltage_list].sum(axis=1)/len(Stack_voltage_list)
            df_polcurve1['Avg_V'] = df_polcurve1[Stack_voltage_list].mean(axis=1)
            df_polcurve1['Max_V'] = df_polcurve1[Stack_voltage_list].max(axis=1)
            df_polcurve1['Min_V'] = df_polcurve1[Stack_voltage_list].min(axis=1)
    #         df_polcurve1.to_csv('df_polcurve1.csv')
            fig.add_trace(go.Scatter(
                                 x = df_polcurve1['CURRENT DENSITY'],
                                 y = df_polcurve1['Avg_V'],
                                 showlegend = True,
                                 mode = 'lines',
                                 name = n,
                                 yaxis = 'y1',

                                 error_y = dict(
                                     type = 'data',
                                     symmetric=False,
                                     visible = True,
                                     array = df_polcurve1['Max_V'] - df_polcurve1['Avg_V'],
                                     arrayminus = df_polcurve1['Avg_V'] - df_polcurve1['Min_V']
                                     ),

                                 customdata = np.stack((df_polcurve1['Min_V'], df_polcurve1['Max_V']), axis=-1),
        #                          customdata2= np.stack(df_polcurve1['Date']), 
                                 hovertemplate ='Max: %{customdata[1]:.2f}'+'Avg: %{y:.2f}'+'<br>'+'Min: %{customdata[0]:.2f}'
        #         +'<br>'+'Date: %{customdata2[0]:dd/mm/yyyy hh:mm:ss'
                                ))
            fig.update_xaxes(title = 'Current Density (A/c'+'m\u00b2)')
            fig.update_yaxes(title = 'Voltage (V)')

            fig.update_layout(title=dict(text="Pol. Curve (Avg of all running stacks)", font=dict(size=20)),
                              xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
                              yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)))
    fig.show()
    # fig.write_image('{}/ALL_Pol_Curve_Avg_all_stacks.png'.format(image_folder_name), engine="orca")
    fig.write_html('{}/ALL_Pol_Curve_Avg_all_stacks.html'.format(image_folder_name), include_plotlyjs=('cdn'))

    # make 3 selected polcurve plot
    first_trace_index = 0
    last_trace_index = len(fig.data)-1
    middle_trace_index = len(fig.data)//2
    selected_traces = []
    for i, trace in enumerate(fig.data):
        if i == first_trace_index or i == middle_trace_index or i == last_trace_index:
            selected_traces.append(trace)
    fig_selected_pc = go.Figure(data=selected_traces)
    fig_selected_pc.layout = fig.layout
    fig_selected_pc.show()
    fig_selected_pc.write_image('{}/Pol_Curve_Avg_all_stacks.png'.format(image_folder_name), engine="orca")
    fig_selected_pc.write_html('{}/Pol_Curve_Avg_all_stacks.html'.format(image_folder_name), include_plotlyjs=('cdn'))

    ################################################################################################################################

    cell_range = {'U1':List_C1_1_C4_16, 'U2':List_C5_1_C8_16, 'L2':List_C9_1_C12_16, 'L1':List_C13_1_C16_16}
    running_cell_positions = {}
    for position, cell in cell_range.items():
        #check if all the cols for a stack is null completely, then pass(do not publish)   
        is_cols_blank = Vcell_data[cell].isnull().all()
        if is_cols_blank.all():
            pass
        else:
            running_cell_positions[position] = cell
    print("running_cell_positions are below")        
    print(str(running_cell_positions)) 

    for m,i in running_cell_positions.items():    
            n = N 
            t1 = T1
            t2 = T2
            fig = go.Figure()
            for n,t1,t2 in zip(n,t1,t2):
                pol_duration_df = Vcell_data[(Vcell_data.Date >= t1) & (Vcell_data.Date <= t2)]
                poldata_list = i.copy()
                poldata_list.extend(['Date','CURRENT SP','STACK CURRENT','CURRENT DENSITY'])
                pol_data = pol_duration_df[poldata_list]
                indexes = pol_data[pol_data.groupby("CURRENT SP")["CURRENT SP"].transform('size') == 1].index
                pol_data.drop(indexes, axis=0, inplace=True)

                df_polcurve1 = pd.DataFrame(columns = list(pol_data.columns))
                list_curr_sp = pol_data['CURRENT SP'].unique()
                for j in list_curr_sp:
                    df_temp = pol_data[pol_data['CURRENT SP']==j]
                    df_polcurve1 = df_polcurve1.append(df_temp.iloc[-2])
                df_polcurve1 = df_polcurve1.reset_index(drop=True)    
    #             print(df_polcurve1)   
                ################################## added logic to remove decreasing CD
                ##create a list out of sc column
                polcurve1_SC = df_polcurve1['STACK CURRENT']
                polcurve1_SC = list(polcurve1_SC)
                # check if next element of list is decreasing , remove it
                index = 0
                while index < (len(polcurve1_SC)-1):
                    diff = polcurve1_SC[index + 1] - polcurve1_SC[index]
                    if diff < 3:
                        polcurve1_SC.pop(index + 1)  # Remove the next element
                    else:
                        index += 1  # Move to the next element if no removal was made
                ## writing a condition that if unique data pnts in the polcurve data is less than 3, then drop that polcurvre and continue to next polcurve        
                if len(polcurve1_SC)<3:
    #                 N.remove(n)
    #                 T1.remove(t1)
    #                 T2.remove(t2)
                    continue
                # else go for making the polcurve further    
                else:               
                    # create a new dataframe out of polcurve1_SC list and join with the original df to pick required data rows.
                    df_polcurve2 = pd.DataFrame()
                    df_polcurve2['STACK CURRENT'] =  polcurve1_SC
                    ## joining two dfs
                    df_polcurve1 = df_polcurve2.merge(df_polcurve1, on='STACK CURRENT', how='left')
                    ##################################    

                    df_polcurve1['Avg_V'] = df_polcurve1[i].mean(axis=1)
                    df_polcurve1['Max_V'] = df_polcurve1[i].max(axis=1)
                    df_polcurve1['Min_V'] = df_polcurve1[i].min(axis=1)
    #                 df_polcurve1.to_csv(m+'_polcurve.csv')
                   # if there is no data (may be stacks removed), then do not plot the trace or legend
                    if df_polcurve1['Avg_V'].any():
                        fig.add_trace(go.Scatter(
                                             x = df_polcurve1['CURRENT DENSITY'],
                                             y = df_polcurve1['Avg_V'],
                                             mode = 'lines',
                                             showlegend = True,
                                             name = n,
                                             yaxis = 'y1',
                                             error_y = dict(
                                                 type = 'data',
                                                 symmetric=False,
                                                 visible = True,
                                                 array = df_polcurve1['Max_V'] - df_polcurve1['Avg_V'],
                                                 arrayminus = df_polcurve1['Avg_V'] - df_polcurve1['Min_V']
                                                 ),

                                             customdata = np.stack((df_polcurve1['Min_V'], df_polcurve1['Max_V']), axis=-1),
                                             hovertemplate ='Max: %{customdata[1]:.2f}'+'Avg: %{y:.2f}'+'<br>'+'Min: %{customdata[0]:.2f}'
                                            ))
                        fig.update_xaxes(title = 'Current Density (A/c'+'m\u00b2)')
                        fig.update_yaxes(title = 'Voltage (V)')

                        fig.update_layout(title=dict(text=m+" Pol Curve", font=dict(size=20)),
                                          xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
                                          yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)))
                    else:
                        fig.add_trace(go.Scatter(
                                             x = df_polcurve1['CURRENT DENSITY'],
                                             y = df_polcurve1['Avg_V'],
                                             mode = 'lines',
                                             showlegend = False))

#             fig.show()
    #         fig.write_image('{}/{}_ALL_pol_curve.png'.format(image_folder_name, m), engine="orca")
            fig.write_html('{}/{}_ALL_pol_curve.html'.format(image_folder_name, m), include_plotlyjs=('cdn'))

            # make 3 selected polcurve plot
            first_trace_index = 0
            last_trace_index = len(fig.data)-1
            middle_trace_index = len(fig.data)//2
            selected_traces = []
            for i, trace in enumerate(fig.data):
                if i == first_trace_index or i == middle_trace_index or i == last_trace_index:
                    selected_traces.append(trace)
            fig_selected_pc = go.Figure(data=selected_traces)
            fig_selected_pc.layout = fig.layout
            fig_selected_pc.show()
            fig_selected_pc.write_image('{}/{}_pol_curve.png'.format(image_folder_name,m), engine="orca")
            fig_selected_pc.write_html('{}/{}_pol_curve.html'.format(image_folder_name,m), include_plotlyjs=('cdn')) 

    # Create min max cell voltage table for pol curves
    n = [N_temp[0], N_temp[len(N_temp)//2], N_temp[len(N_temp)-1]]
    t1=[T1_temp[0], T1_temp[len(N_temp)//2], T1_temp[len(N_temp)-1]]
    t2=[T2_temp[0], T2_temp[len(N_temp)//2], T2_temp[len(N_temp)-1]]

    MinMax_cell_df = pd.DataFrame()
    Pol_curve_duration = []
    Max_Current_density = []
    Max_V_cell = []
    Min_V_cell = []

    for n,t1,t2 in zip(n,t1,t2):
        pol_duration_df = Vcell_data[(Vcell_data.Date >= t1) & (Vcell_data.Date <= t2)]

        Pol_curve_duration.append(n)
        Max_Current_density.append(round(max(pol_duration_df['CURRENT DENSITY']),2))

        pol_df = Vcell_data[Vcell_data['Date'] == t1][Stack_voltage_list]

        max_v_cell = pol_df.idxmax(axis=1).values
        max_v_cell_str = ", ".join(str(num) for num in max_v_cell)

        min_v_cell = pol_df.idxmin(axis=1).values
        min_v_cell_str = ", ".join(str(num) for num in min_v_cell)

        Max_V_cell.append(max_v_cell_str)
        Min_V_cell.append(min_v_cell_str)

    MinMax_cell_df['Pol curve duration'] = Pol_curve_duration
    MinMax_cell_df['Max Current density (A/cm2)'] = Max_Current_density
    MinMax_cell_df['Max_V_cell'] = Max_V_cell
    MinMax_cell_df['Min_V_cell'] = Min_V_cell

    # MinMax_cell_df

    ######### converting the table dataframe in a cropped image
    header_values = ['<b>' + col + '</b>' for col in MinMax_cell_df.columns]

    fig = go.Figure(data=[go.Table(
        header=dict(values = header_values,
    #                 fill_color='paleturquoise',
                    fill_color='lightgray',
                    align='center',
                    line=dict(color='black', width=1),
                    font=dict(color='black', size=12)
                   ), 
        cells=dict(values=[MinMax_cell_df['Pol curve duration'],
                        MinMax_cell_df['Max Current density (A/cm2)'], MinMax_cell_df['Max_V_cell'], MinMax_cell_df['Min_V_cell']],
                   fill_color='lavender',
                   align='center',
                   line=dict(color='black', width=1)))
                         ])

    fig.update_layout(
        margin=dict(t=0, b=0, l=0, r=0),
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        showlegend=False
    )

    # fig.write_image('./images/MinMax_cell.png')
    fig.write_image('{}/MinMax_cell.png'.format(image_folder_name), engine="orca")

    # Open the image
    image = Image.open("./"+image_folder_name + "/"+"MinMax_cell.png")

    # Define the crop region (left, upper, right, lower)
    crop_region = (0, 0, 700, 108)

    # Crop the image
    MinMax_cell = image.crop(crop_region)

    filename = "MinMax_cell.png"
    folder_path = './'+ image_folder_name + '/'

    MinMax_cell.save(folder_path + filename)
    print("Polcurves are exported")
    
# if t_start/t_end is empty, then do not run the pol curve function
if len(t_start) == 0:
    pass
    print('No pol curve in the data')
else:
    pol_curve_func()

[Timestamp('2023-07-10 15:57:05'), Timestamp('2023-07-10 16:43:55'), Timestamp('2023-07-10 19:26:10'), Timestamp('2023-07-10 19:38:55'), Timestamp('2023-07-10 19:48:30'), Timestamp('2023-07-10 21:30:40'), Timestamp('2023-07-10 21:39:35'), Timestamp('2023-07-10 22:51:50'), Timestamp('2023-07-11 08:28:10'), Timestamp('2023-07-11 12:47:45'), Timestamp('2023-07-12 07:35:30'), Timestamp('2023-07-12 08:29:10'), Timestamp('2023-07-12 11:40:45'), Timestamp('2023-07-13 08:13:00'), Timestamp('2023-07-13 10:34:00'), Timestamp('2023-07-13 13:43:00'), Timestamp('2023-07-13 14:16:00'), Timestamp('2023-07-13 15:53:00'), Timestamp('2023-07-13 20:19:00'), Timestamp('2023-07-13 23:33:00'), Timestamp('2023-07-14 03:04:35'), Timestamp('2023-07-14 13:21:00'), Timestamp('2023-07-14 18:00:45'), Timestamp('2023-07-14 18:49:40'), Timestamp('2023-07-15 01:06:50'), Timestamp('2023-07-15 09:00:05'), Timestamp('2023-07-15 09:52:15'), Timestamp('2023-07-15 12:36:55'), Timestamp('2023-07-15 13:07:35'), Timestamp('20

running_cell_positions are below
{'U1': ['C1-1', 'C1-2', 'C1-3', 'C1-4', 'C1-5', 'C1-6', 'C1-7', 'C1-8', 'C1-9', 'C1-10', 'C1-11', 'C1-12', 'C1-13', 'C1-14', 'C1-15', 'C1-16', 'C2-1', 'C2-2', 'C2-3', 'C2-4', 'C2-5', 'C2-6', 'C2-7', 'C2-8', 'C2-9', 'C2-10', 'C2-11', 'C2-12', 'C2-13', 'C2-14', 'C2-15', 'C2-16', 'C3-1', 'C3-2', 'C3-3', 'C3-4', 'C3-5', 'C3-6', 'C3-7', 'C3-8', 'C3-9', 'C3-10', 'C3-11', 'C3-12', 'C3-13', 'C3-14', 'C3-15', 'C3-16', 'C4-1', 'C4-2', 'C4-3', 'C4-4', 'C4-5', 'C4-6', 'C4-7', 'C4-8'], 'U2': ['C5-1', 'C5-2', 'C5-3', 'C5-4', 'C5-5', 'C5-6', 'C5-7', 'C5-8', 'C5-9', 'C5-10', 'C5-11', 'C5-12', 'C5-13', 'C5-14', 'C5-15', 'C5-16', 'C6-1', 'C6-2', 'C6-3', 'C6-4', 'C6-5', 'C6-6', 'C6-7', 'C6-8', 'C6-9', 'C6-10', 'C6-11', 'C6-12', 'C6-13', 'C6-14', 'C6-15', 'C6-16', 'C7-1', 'C7-2', 'C7-3', 'C7-4', 'C7-5', 'C7-6', 'C7-7', 'C7-8', 'C7-9', 'C7-10', 'C7-11', 'C7-12', 'C7-13', 'C7-14', 'C7-15', 'C7-16', 'C8-1', 'C8-2', 'C8-3', 'C8-4', 'C8-5', 'C8-6', 'C8-7', 'C8-8'], 'L2': ['C9-1

Polcurves are exported


In [17]:
# ###################################### POL CURVE CREATION ######################################################################
# def pol_curve_func():
#     Stack_voltage_list = [
#         'C1-1', 'C1-2', 'C1-3', 'C1-4', 'C1-5', 'C1-6', 'C1-7', 'C1-8', 'C1-9', 'C1-10', 'C1-11', 'C1-12', 'C1-13', 'C1-14', 'C1-15', 'C1-16', 'C2-1', 'C2-2', 'C2-3', 'C2-4', 'C2-5', 'C2-6', 'C2-7', 'C2-8', 'C2-9', 'C2-10', 'C2-11', 'C2-12', 'C2-13', 'C2-14', 'C2-15', 'C2-16', 'C3-1', 'C3-2', 'C3-3', 'C3-4', 'C3-5', 'C3-6', 'C3-7', 'C3-8', 'C3-9', 'C3-10', 'C3-11', 'C3-12', 'C3-13', 'C3-14', 'C3-15', 'C3-16', 'C4-1', 'C4-2', 'C4-3', 'C4-4', 'C4-5', 'C4-6', 'C4-7', 'C4-8'
#     #                       , 'C4-9', 'C4-10', 'C4-11', 'C4-12', 'C4-13', 'C4-14', 'C4-15', 'C4-16',
#         ,'C5-1', 'C5-2', 'C5-3', 'C5-4', 'C5-5', 'C5-6', 'C5-7', 'C5-8', 'C5-9', 'C5-10', 'C5-11', 'C5-12', 'C5-13', 'C5-14', 'C5-15', 'C5-16', 'C6-1', 'C6-2', 'C6-3', 'C6-4', 'C6-5', 'C6-6', 'C6-7', 'C6-8', 'C6-9', 'C6-10', 'C6-11', 'C6-12', 'C6-13', 'C6-14', 'C6-15', 'C6-16', 'C7-1', 'C7-2', 'C7-3', 'C7-4', 'C7-5', 'C7-6', 'C7-7', 'C7-8', 'C7-9', 'C7-10', 'C7-11', 'C7-12', 'C7-13', 'C7-14', 'C7-15', 'C7-16', 'C8-1', 'C8-2', 'C8-3', 'C8-4', 'C8-5', 'C8-6', 'C8-7', 'C8-8'
#     #                       , 'C8-9', 'C8-10', 'C8-11', 'C8-12', 'C8-13', 'C8-14', 'C8-15', 'C8-16',
#         ,'C9-1', 'C9-2', 'C9-3', 'C9-4', 'C9-5', 'C9-6', 'C9-7', 'C9-8', 'C9-9', 'C9-10', 'C9-11', 'C9-12', 'C9-13', 'C9-14', 'C9-15', 'C9-16', 'C10-1', 'C10-2', 'C10-3', 'C10-4', 'C10-5', 'C10-6', 'C10-7', 'C10-8', 'C10-9', 'C10-10', 'C10-11', 'C10-12', 'C10-13', 'C10-14', 'C10-15', 'C10-16', 'C11-1', 'C11-2', 'C11-3', 'C11-4', 'C11-5', 'C11-6', 'C11-7', 'C11-8', 'C11-9', 'C11-10', 'C11-11', 'C11-12', 'C11-13', 'C11-14', 'C11-15', 'C11-16','C12-1', 'C12-2', 'C12-3', 'C12-4', 'C12-5', 'C12-6', 'C12-7', 'C12-8'
#     #                       , 'C12-9', 'C12-10', 'C12-11', 'C12-12', 'C12-13', 'C12-14', 'C12-15', 'C12-16',
#         ,'C13-1', 'C13-2', 'C13-3', 'C13-4', 'C13-5', 'C13-6', 'C13-7', 'C13-8', 'C13-9', 'C13-10', 'C13-11', 'C13-12', 'C13-13', 'C13-14', 'C13-15', 'C13-16', 'C14-1', 'C14-2', 'C14-3', 'C14-4', 'C14-5', 'C14-6', 'C14-7', 'C14-8', 'C14-9', 'C14-10', 'C14-11', 'C14-12', 'C14-13', 'C14-14', 'C14-15', 'C14-16', 'C15-1', 'C15-2', 'C15-3', 'C15-4', 'C15-5', 'C15-6', 'C15-7', 'C15-8', 'C15-9', 'C15-10', 'C15-11', 'C15-12', 'C15-13', 'C15-14', 'C15-15', 'C15-16', 'C16-1', 'C16-2', 'C16-3', 'C16-4', 'C16-5', 'C16-6', 'C16-7', 'C16-8'
#     #              , 'C16-9', 'C16-10', 'C16-11', 'C16-12', 'C16-13', 'C16-14', 'C16-15', 'C16-16'
#     ]

#     n=N
#     t1=T1
#     t2=T2

#     #creating lists to later make Min/Max Cell table from these
#     N_temp=N.copy()
#     T1_temp=t1.copy()
#     T2_temp=t2.copy()

#     fig = go.Figure()

#     for n,t1,t2 in zip(n,t1,t2):

#         pol_duration_df = Vcell_data[(Vcell_data.Date >= t1) & (Vcell_data.Date <= t2)]
#         poldata_list = Stack_voltage_list.copy()
#         poldata_list.extend(['Date','CURRENT SP','STACK CURRENT','CURRENT DENSITY'])
#         pol_data = pol_duration_df[poldata_list]

#         ## choose indexes that have only one row for a current SP  
#         indexes = pol_data[pol_data.groupby("CURRENT SP")["CURRENT SP"].transform('size') == 1].index
#         ## and then drop those rows
#         pol_data.drop(indexes, axis=0, inplace=True)
#         df_polcurve1 = pd.DataFrame(columns = list(pol_data.columns))
#         list_curr_sp = pol_data['CURRENT SP'].unique()
#         for i in list_curr_sp:
#             df_temp = pol_data[pol_data['CURRENT SP']==i]
#             df_polcurve1 = df_polcurve1.append(df_temp.iloc[-2])    
#         df_polcurve1 = df_polcurve1.reset_index(drop=True)    
#         ############################# added logic to remove decreasing CD
#         ##create a list out of sc column
#         polcurve1_SC = df_polcurve1['STACK CURRENT']
#         polcurve1_SC = list(polcurve1_SC)
#         # check if next element of list is decreasing , remove it
#         index = 0
#         while index < (len(polcurve1_SC)-1):
#             diff = polcurve1_SC[index + 1] - polcurve1_SC[index]
#             if diff < 3:
#                 polcurve1_SC.pop(index + 1)  # Remove the next element
#             else:
#                 index += 1  # Move to the next element if no removal was made

#         ## writing a condition that if unique data pnts in the polcurve data is less than 3, then drop that polcurvre and continue to next polcurve        
#         if len(polcurve1_SC)<3:
#             N_temp.remove(n)
#             T1_temp.remove(t1)
#             T2_temp.remove(t2)
#             continue
#         # else go for making the polcurve further    
#         else:                
#             # create a new dataframe out of polcurve1_SC list and join with the original df to pick required data rows.
#             df_polcurve2 = pd.DataFrame()
#             df_polcurve2['STACK CURRENT'] =  polcurve1_SC
#             ## joining two dfs
#             df_polcurve1 = df_polcurve2.merge(df_polcurve1, on='STACK CURRENT', how='left')
#             ###############################
#         #     print(df_polcurve1)
#     #         df_polcurve1['Avg_V'] = df_polcurve1[Stack_voltage_list].sum(axis=1)/len(Stack_voltage_list)
#             df_polcurve1['Avg_V'] = df_polcurve1[Stack_voltage_list].mean(axis=1)
#             df_polcurve1['Max_V'] = df_polcurve1[Stack_voltage_list].max(axis=1)
#             df_polcurve1['Min_V'] = df_polcurve1[Stack_voltage_list].min(axis=1)
#     #         df_polcurve1.to_csv('df_polcurve1.csv')
#             fig.add_trace(go.Scatter(
#                                  x = df_polcurve1['CURRENT DENSITY'],
#                                  y = df_polcurve1['Avg_V'],
#                                  showlegend = True,
#                                  mode = 'lines',
#                                  name = n,
#                                  yaxis = 'y1',

#                                  error_y = dict(
#                                      type = 'data',
#                                      symmetric=False,
#                                      visible = True,
#                                      array = df_polcurve1['Max_V'] - df_polcurve1['Avg_V'],
#                                      arrayminus = df_polcurve1['Avg_V'] - df_polcurve1['Min_V']
#                                      ),

#                                  customdata = np.stack((df_polcurve1['Min_V'], df_polcurve1['Max_V']), axis=-1),
#         #                          customdata2= np.stack(df_polcurve1['Date']), 
#                                  hovertemplate ='Max: %{customdata[1]:.2f}'+'Avg: %{y:.2f}'+'<br>'+'Min: %{customdata[0]:.2f}'
#         #         +'<br>'+'Date: %{customdata2[0]:dd/mm/yyyy hh:mm:ss'
#                                 ))
#             fig.update_xaxes(title = 'Current Density (A/c'+'m\u00b2)')
#             fig.update_yaxes(title = 'Voltage (V)')

#             fig.update_layout(title=dict(text="Pol. Curve (Avg of all running stacks)", font=dict(size=20)),
#                               xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
#                               yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)))
#     fig.show()
#     # fig.write_image('{}/ALL_Pol_Curve_Avg_all_stacks.png'.format(image_folder_name), engine="orca")
#     fig.write_html('{}/ALL_Pol_Curve_Avg_all_stacks.html'.format(image_folder_name), include_plotlyjs=('cdn'))

#     # make 3 selected polcurve plot
#     first_trace_index = 0
#     last_trace_index = len(fig.data)-1
#     middle_trace_index = len(fig.data)//2
#     selected_traces = []
#     for i, trace in enumerate(fig.data):
#         if i == first_trace_index or i == middle_trace_index or i == last_trace_index:
#             selected_traces.append(trace)
#     fig_selected_pc = go.Figure(data=selected_traces)
#     fig_selected_pc.layout = fig.layout
#     fig_selected_pc.show()
#     fig_selected_pc.write_image('{}/Pol_Curve_Avg_all_stacks.png'.format(image_folder_name), engine="orca")
#     fig_selected_pc.write_html('{}/Pol_Curve_Avg_all_stacks.html'.format(image_folder_name), include_plotlyjs=('cdn'))

#     ################################################################################################################################

#     cell_range = {'U1':List_C1_1_C4_16, 'U2':List_C5_1_C8_16, 'L2':List_C9_1_C12_16, 'L1':List_C13_1_C16_16}
#     running_cell_positions = {}
#     for position, cell in cell_range.items():
#         #check if all the cols for a stack is null completely, then pass(do not publish)   
#         is_cols_blank = Vcell_data[cell].isnull().all()
#         if is_cols_blank.all():
#             pass
#         else:
#             running_cell_positions[position] = cell
#     print("running_cell_positions are below")        
#     print(str(running_cell_positions)) 

#     for m,i in running_cell_positions.items():    
#             n = N 
#             t1 = T1
#             t2 = T2
#             fig = go.Figure()
#             for n,t1,t2 in zip(n,t1,t2):
#                 pol_duration_df = Vcell_data[(Vcell_data.Date >= t1) & (Vcell_data.Date <= t2)]
#                 poldata_list = i.copy()
#                 poldata_list.extend(['Date','CURRENT SP','STACK CURRENT','CURRENT DENSITY'])
#                 pol_data = pol_duration_df[poldata_list]
#                 indexes = pol_data[pol_data.groupby("CURRENT SP")["CURRENT SP"].transform('size') == 1].index
#                 pol_data.drop(indexes, axis=0, inplace=True)

#                 df_polcurve1 = pd.DataFrame(columns = list(pol_data.columns))
#                 list_curr_sp = pol_data['CURRENT SP'].unique()
#                 for j in list_curr_sp:
#                     df_temp = pol_data[pol_data['CURRENT SP']==j]
#                     df_polcurve1 = df_polcurve1.append(df_temp.iloc[-2])
#                 df_polcurve1 = df_polcurve1.reset_index(drop=True)    
#     #             print(df_polcurve1)   
#                 ################################## added logic to remove decreasing CD
#                 ##create a list out of sc column
#                 polcurve1_SC = df_polcurve1['STACK CURRENT']
#                 polcurve1_SC = list(polcurve1_SC)
#                 # check if next element of list is decreasing , remove it
#                 index = 0
#                 while index < (len(polcurve1_SC)-1):
#                     diff = polcurve1_SC[index + 1] - polcurve1_SC[index]
#                     if diff < 3:
#                         polcurve1_SC.pop(index + 1)  # Remove the next element
#                     else:
#                         index += 1  # Move to the next element if no removal was made
#                 ## writing a condition that if unique data pnts in the polcurve data is less than 3, then drop that polcurvre and continue to next polcurve        
#                 if len(polcurve1_SC)<3:
#     #                 N.remove(n)
#     #                 T1.remove(t1)
#     #                 T2.remove(t2)
#                     continue
#                 # else go for making the polcurve further    
#                 else:               
#                     # create a new dataframe out of polcurve1_SC list and join with the original df to pick required data rows.
#                     df_polcurve2 = pd.DataFrame()
#                     df_polcurve2['STACK CURRENT'] =  polcurve1_SC
#                     ## joining two dfs
#                     df_polcurve1 = df_polcurve2.merge(df_polcurve1, on='STACK CURRENT', how='left')
#                     ##################################    

#                     df_polcurve1['Avg_V'] = df_polcurve1[i].mean(axis=1)
#                     df_polcurve1['Max_V'] = df_polcurve1[i].max(axis=1)
#                     df_polcurve1['Min_V'] = df_polcurve1[i].min(axis=1)
#     #                 df_polcurve1.to_csv(m+'_polcurve.csv')
#                    # if there is no data (may be stacks removed), then do not plot the trace or legend
#                     if df_polcurve1['Avg_V'].any():
#                         fig.add_trace(go.Scatter(
#                                              x = df_polcurve1['CURRENT DENSITY'],
#                                              y = df_polcurve1['Avg_V'],
#                                              mode = 'lines',
#                                              showlegend = True,
#                                              name = n,
#                                              yaxis = 'y1',
#                                              error_y = dict(
#                                                  type = 'data',
#                                                  symmetric=False,
#                                                  visible = True,
#                                                  array = df_polcurve1['Max_V'] - df_polcurve1['Avg_V'],
#                                                  arrayminus = df_polcurve1['Avg_V'] - df_polcurve1['Min_V']
#                                                  ),

#                                              customdata = np.stack((df_polcurve1['Min_V'], df_polcurve1['Max_V']), axis=-1),
#                                              hovertemplate ='Max: %{customdata[1]:.2f}'+'Avg: %{y:.2f}'+'<br>'+'Min: %{customdata[0]:.2f}'
#                                             ))
#                         fig.update_xaxes(title = 'Current Density (A/c'+'m\u00b2)')
#                         fig.update_yaxes(title = 'Voltage (V)')

#                         fig.update_layout(title=dict(text=m+" Pol Curve", font=dict(size=20)),
#                                           xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
#                                           yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)))
#                     else:
#                         fig.add_trace(go.Scatter(
#                                              x = df_polcurve1['CURRENT DENSITY'],
#                                              y = df_polcurve1['Avg_V'],
#                                              mode = 'lines',
#                                              showlegend = False))

# #             fig.show()
#     #         fig.write_image('{}/{}_ALL_pol_curve.png'.format(image_folder_name, m), engine="orca")
#             fig.write_html('{}/{}_ALL_pol_curve.html'.format(image_folder_name, m), include_plotlyjs=('cdn'))

#             # make 3 selected polcurve plot
#             first_trace_index = 0
#             last_trace_index = len(fig.data)-1
#             middle_trace_index = len(fig.data)//2
#             selected_traces = []
#             for i, trace in enumerate(fig.data):
#                 if i == first_trace_index or i == middle_trace_index or i == last_trace_index:
#                     selected_traces.append(trace)
#             fig_selected_pc = go.Figure(data=selected_traces)
#             fig_selected_pc.layout = fig.layout
#             fig_selected_pc.show()
#             fig_selected_pc.write_image('{}/{}_pol_curve.png'.format(image_folder_name,m), engine="orca")
#             fig_selected_pc.write_html('{}/{}_pol_curve.html'.format(image_folder_name,m), include_plotlyjs=('cdn')) 

#     # Create min max cell voltage table for pol curves
#     n = [N_temp[0], N_temp[len(N_temp)//2], N_temp[len(N_temp)-1]]
#     t1=[T1_temp[0], T1_temp[len(N_temp)//2], T1_temp[len(N_temp)-1]]
#     t2=[T2_temp[0], T2_temp[len(N_temp)//2], T2_temp[len(N_temp)-1]]

#     MinMax_cell_df = pd.DataFrame()
#     Pol_curve_duration = []
#     Max_Current_density = []
#     Max_V_cell = []
#     Min_V_cell = []

#     for n,t1,t2 in zip(n,t1,t2):
#         pol_duration_df = Vcell_data[(Vcell_data.Date >= t1) & (Vcell_data.Date <= t2)]

#         Pol_curve_duration.append(n)
#         Max_Current_density.append(round(max(pol_duration_df['CURRENT DENSITY']),2))

#         pol_df = Vcell_data[Vcell_data['Date'] == t1][Stack_voltage_list]

#         max_v_cell = pol_df.idxmax(axis=1).values
#         max_v_cell_str = ", ".join(str(num) for num in max_v_cell)

#         min_v_cell = pol_df.idxmin(axis=1).values
#         min_v_cell_str = ", ".join(str(num) for num in min_v_cell)

#         Max_V_cell.append(max_v_cell_str)
#         Min_V_cell.append(min_v_cell_str)

#     MinMax_cell_df['Pol curve duration'] = Pol_curve_duration
#     MinMax_cell_df['Max Current density (A/cm2)'] = Max_Current_density
#     MinMax_cell_df['Max_V_cell'] = Max_V_cell
#     MinMax_cell_df['Min_V_cell'] = Min_V_cell

#     # MinMax_cell_df

#     ######### converting the table dataframe in a cropped image
#     header_values = ['<b>' + col + '</b>' for col in MinMax_cell_df.columns]

#     fig = go.Figure(data=[go.Table(
#         header=dict(values = header_values,
#     #                 fill_color='paleturquoise',
#                     fill_color='lightgray',
#                     align='center',
#                     line=dict(color='black', width=1),
#                     font=dict(color='black', size=12)
#                    ), 
#         cells=dict(values=[MinMax_cell_df['Pol curve duration'],
#                         MinMax_cell_df['Max Current density (A/cm2)'], MinMax_cell_df['Max_V_cell'], MinMax_cell_df['Min_V_cell']],
#                    fill_color='lavender',
#                    align='center',
#                    line=dict(color='black', width=1)))
#                          ])

#     fig.update_layout(
#         margin=dict(t=0, b=0, l=0, r=0),
#         xaxis=dict(visible=False),
#         yaxis=dict(visible=False),
#         showlegend=False
#     )

#     # fig.write_image('./images/MinMax_cell.png')
#     fig.write_image('{}/MinMax_cell.png'.format(image_folder_name), engine="orca")

#     # Open the image
#     image = Image.open("./"+image_folder_name + "/"+"MinMax_cell.png")

#     # Define the crop region (left, upper, right, lower)
#     crop_region = (0, 0, 700, 108)

#     # Crop the image
#     MinMax_cell = image.crop(crop_region)

#     filename = "MinMax_cell.png"
#     folder_path = './'+ image_folder_name + '/'

#     MinMax_cell.save(folder_path + filename)

#     print("Polcurves are exported")
        
# # if t_start/t_end is empty, then do not run the pol curve function
# if len(t_start) == 0:
#     pass
# else:
#     pol_curve_func()    

In [18]:
## create SSE chart 
# Stack_voltage_list = [
#     'C1-1', 'C1-2', 'C1-3', 'C1-4', 'C1-5', 'C1-6', 'C1-7', 'C1-8', 'C1-9', 'C1-10', 'C1-11', 'C1-12', 'C1-13', 'C1-14', 'C1-15', 'C1-16', 'C2-1', 'C2-2', 'C2-3', 'C2-4', 'C2-5', 'C2-6', 'C2-7', 'C2-8', 'C2-9', 'C2-10', 'C2-11', 'C2-12', 'C2-13', 'C2-14', 'C2-15', 'C2-16', 'C3-1', 'C3-2', 'C3-3', 'C3-4', 'C3-5', 'C3-6', 'C3-7', 'C3-8', 'C3-9', 'C3-10', 'C3-11', 'C3-12', 'C3-13', 'C3-14', 'C3-15', 'C3-16', 'C4-1', 'C4-2', 'C4-3', 'C4-4', 'C4-5', 'C4-6', 'C4-7', 'C4-8'
# #                       , 'C4-9', 'C4-10', 'C4-11', 'C4-12', 'C4-13', 'C4-14', 'C4-15', 'C4-16',
#     ,'C5-1', 'C5-2', 'C5-3', 'C5-4', 'C5-5', 'C5-6', 'C5-7', 'C5-8', 'C5-9', 'C5-10', 'C5-11', 'C5-12', 'C5-13', 'C5-14', 'C5-15', 'C5-16', 'C6-1', 'C6-2', 'C6-3', 'C6-4', 'C6-5', 'C6-6', 'C6-7', 'C6-8', 'C6-9', 'C6-10', 'C6-11', 'C6-12', 'C6-13', 'C6-14', 'C6-15', 'C6-16', 'C7-1', 'C7-2', 'C7-3', 'C7-4', 'C7-5', 'C7-6', 'C7-7', 'C7-8', 'C7-9', 'C7-10', 'C7-11', 'C7-12', 'C7-13', 'C7-14', 'C7-15', 'C7-16', 'C8-1', 'C8-2', 'C8-3', 'C8-4', 'C8-5', 'C8-6', 'C8-7', 'C8-8'
# #                       , 'C8-9', 'C8-10', 'C8-11', 'C8-12', 'C8-13', 'C8-14', 'C8-15', 'C8-16',
#     ,'C9-1', 'C9-2', 'C9-3', 'C9-4', 'C9-5', 'C9-6', 'C9-7', 'C9-8', 'C9-9', 'C9-10', 'C9-11', 'C9-12', 'C9-13', 'C9-14', 'C9-15', 'C9-16', 'C10-1', 'C10-2', 'C10-3', 'C10-4', 'C10-5', 'C10-6', 'C10-7', 'C10-8', 'C10-9', 'C10-10', 'C10-11', 'C10-12', 'C10-13', 'C10-14', 'C10-15', 'C10-16', 'C11-1', 'C11-2', 'C11-3', 'C11-4', 'C11-5', 'C11-6', 'C11-7', 'C11-8', 'C11-9', 'C11-10', 'C11-11', 'C11-12', 'C11-13', 'C11-14', 'C11-15', 'C11-16','C12-1', 'C12-2', 'C12-3', 'C12-4', 'C12-5', 'C12-6', 'C12-7', 'C12-8'
# #                       , 'C12-9', 'C12-10', 'C12-11', 'C12-12', 'C12-13', 'C12-14', 'C12-15', 'C12-16',
#     ,'C13-1', 'C13-2', 'C13-3', 'C13-4', 'C13-5', 'C13-6', 'C13-7', 'C13-8', 'C13-9', 'C13-10', 'C13-11', 'C13-12', 'C13-13', 'C13-14', 'C13-15', 'C13-16', 'C14-1', 'C14-2', 'C14-3', 'C14-4', 'C14-5', 'C14-6', 'C14-7', 'C14-8', 'C14-9', 'C14-10', 'C14-11', 'C14-12', 'C14-13', 'C14-14', 'C14-15', 'C14-16', 'C15-1', 'C15-2', 'C15-3', 'C15-4', 'C15-5', 'C15-6', 'C15-7', 'C15-8', 'C15-9', 'C15-10', 'C15-11', 'C15-12', 'C15-13', 'C15-14', 'C15-15', 'C15-16', 'C16-1', 'C16-2', 'C16-3', 'C16-4', 'C16-5', 'C16-6', 'C16-7', 'C16-8'
# #              , 'C16-9', 'C16-10', 'C16-11', 'C16-12', 'C16-13', 'C16-14', 'C16-15', 'C16-16'
# ]    

# No_of_Stacks = len(Stack_voltage_list)    
Vcell_data['STACK VOLTAGE'] = Vcell_data[Stack_voltage_list].sum(axis=1)
# print(sys_data[sys_data['Date'] == '05-11-2023 12:26:40'][['Date','STACK VOLTAGE']])

Vcell_data['no_of_stacks'] = Vcell_data[Stack_voltage_list].count(axis=1)

## if no of stacks is not a multiple of 56 , then drop that row
Vcell_data  = Vcell_data[Vcell_data['no_of_stacks']%56 == 0]

# sys_data['STACK SPECIFIC ENERGY'] = sys_data['STACK VOLTAGE']/(0.04*No_of_Stacks) [this was old calculation which was corrected later]
Vcell_data['STACK SPECIFIC ENERGY'] = Vcell_data['STACK VOLTAGE']/(0.0376*Vcell_data['no_of_stacks'])
# print(sys_data[sys_data['Date'] == '05-11-2023 12:26:40'][['Date','STACK SPECIFIC ENERGY']])


Stack_Specific_energy = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['STACK SPECIFIC ENERGY'],
                       mode = 'markers',
                       #marker=dict(size=3),
                       #mode = 'lines',
                       name = 'Stack Specific Energy (kWh/kg)',
                       yaxis = 'y1'
                       #text = df.column (this is the info displayed on hover)
                       )
Current_Density = go.Scattergl(
                       x = Vcell_data['Date'],
                       y = Vcell_data['CURRENT DENSITY'],
                       mode = 'markers',
#                        mode = 'lines',
                       name = 'Current Density (A/c'+'m\u00b2)',
                       yaxis = 'y2'
                       #text = df.column (this is the info displayed on hover)
                       )

# update the data list
data = [Stack_Specific_energy, Current_Density]

# configure the layout
layout = go.Layout(
            title = 'LHC Stack Specific'+'<br>'+'Energy (DC)',
            #to define axis, can either use go.layout.etc or dict(etc)
            xaxis = go.layout.XAxis(
#                                     title = 'Date',
                                    tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                    hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                    showgrid = False,
                                    #range=[100,steady_state['Duration(hr)'].max()]
                                    ),    
            yaxis = dict(title='Stack Specific'+'<br>'+'Energy (kWh/kg)',
                                showgrid = False,
                                rangemode='tozero'),
            yaxis2= dict(title='Current Density'+'<br>' +'(A/c'+'m\u00b2)',
                                 overlaying='y',
                                 side = 'right',
                                 range = [0,2],
                                 showgrid = False,
                                 rangemode='tozero'),
#                                  range=[0,600]),
            legend = dict(xanchor="right",x=1.4,yanchor="top", y =1.3, bgcolor = 'rgba(0,0,0,0)'),
            template = 'simple_white',
            font = dict(size = 18)
            )

# create the figure
fig4 = go.Figure(data = data, layout = layout)

fig4.update_xaxes(tickangle=90)

fig4.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
                  yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
                  yaxis2 = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) 
                  )

fig4.write_image('{}/LHC_Stack_Specific_Energy.png'.format(image_folder_name), engine="orca")
fig4.write_html('{}/LHC_Stack_Specific_Energy.html'.format(image_folder_name), include_plotlyjs=('cdn'))

# fig4.show()

In [19]:
## formatting first two slides 
prs = Presentation('{}.pptx'.format(template_ppt_fn))

## edit first table (only table) of first slide       
# Select the slide containing the table (e.g., slide 0)
slide = prs.slides[0]
# Select the table in the slide (e.g., table 0)
table1 = slide.shapes[3].table
today = date.today()
# write body cells
table1.cell(0, 0).text = "Date    "+str(today.strftime("%d/%m/%Y"))
now = datetime.now()
dt_string = now.strftime("%H:%M") + "hrs.(IST)"
table1.cell(1, 0).text = "Time  "+dt_string

## edit first table of second slide       
slide = prs.slides[1]
table2 = slide.shapes[0].table
table2.cell(0, 6).text = str(int(total_duration_hrs))+" hrs"
table2.cell(1, 5).text = Start_time
table2.cell(1, 7).text = End_time

# ## edit second table of second slide       
# stack_dictionary = {'U1':['','','',''], 'U2':['','','',''], 'L2':['','','',''], 'L1':['','','','91-56']}
# table3 = slide.shapes[1].table
# table3.cell(3, 1).text = stack_dictionary["U1"][3]
# table3.cell(3, 2).text = stack_dictionary["U1"][2]
# table3.cell(3, 3).text = stack_dictionary["U1"][1]
# table3.cell(3, 4).text = stack_dictionary["U1"][0]

# table3.cell(3, 5).text = stack_dictionary["U2"][3]
# table3.cell(3, 6).text = stack_dictionary["U2"][2]
# table3.cell(3, 7).text = stack_dictionary["U2"][1]
# table3.cell(3, 8).text = stack_dictionary["U2"][0]

# table3.cell(3,  9).text = stack_dictionary["L2"][3]
# table3.cell(3, 10).text = stack_dictionary["L2"][2]
# table3.cell(3, 11).text = stack_dictionary["L2"][1]
# table3.cell(3, 12).text = stack_dictionary["L2"][0]

# table3.cell(3, 13).text = stack_dictionary["L1"][3]
# table3.cell(3, 14).text = stack_dictionary["L1"][2]
# table3.cell(3, 15).text = stack_dictionary["L1"][1]
# table3.cell(3, 16).text = stack_dictionary["L1"][0]

# tables_list = [table3]
# for j in tables_list:
#     for row in j.rows:
#         for cell in row.cells:

#             # Set font size
#             cell.text_frame.paragraphs[0].font.size = Pt(14)  # Change the font size to your desired value

#             # Set font style
#             cell.text_frame.paragraphs[0].font.name = 'Quicksand (Headings)'  # Change the font style to your desired font

#             # Set alignment
#             cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER  # Change the alignment to your desired value
            
#             # set color to black
#             cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)  # (R, G, B) = (0, 0, 0) for black
#             cell.text_frame.paragraphs[0].font.bold = False


## formatting all tables
tables_list = [table1, table2]
for j in tables_list:
    for row in j.rows:
        for cell in row.cells:

            # Set font size
            cell.text_frame.paragraphs[0].font.size = Pt(16)  # Change the font size to your desired value

            # Set font style
            cell.text_frame.paragraphs[0].font.name = 'Quicksand (Headings)'  # Change the font style to your desired font

            # Set alignment
            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER  # Change the alignment to your desired value
            
            # set color to black
            cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)  # (R, G, B) = (0, 0, 0) for black
            cell.text_frame.paragraphs[0].font.bold = False

             
# Save the modified PowerPoint presentation
# prs.save('{}.pptx'.format(ppt_fn))

In [20]:
### Publishing the ppt

slide = prs.slides[2]
height = Inches(3.19)
width = Inches(5.04)
row1_y = Inches(0.69)
row2_y = Inches(4.16)
column1_x = Inches(0.69)
column2_x = Inches(7.12)

# place the LHC_Stack_Specific_Energy Chart
pic = slide.shapes.add_picture('{}/LHC_Stack_Specific_Energy.png'.format(image_folder_name), column1_x, row2_y,width=width, height=height)
pic.click_action.hyperlink.address = '{}/LHC_Stack_Specific_Energy.html'.format(image_folder_name)

# place the LHC_TEMP_&_Level_Sensor_Chart
pic = slide.shapes.add_picture('{}/LHC_TEMP_&_Level_Sensor_Chart.png'.format(image_folder_name), column1_x, row1_y,width=width, height=height)
pic.click_action.hyperlink.address = '{}/LHC_TEMP_&_Level_Sensor_Chart.html'.format(image_folder_name)

# place the LHC_O2_&_H2_leak_Sensor_Chart
pic = slide.shapes.add_picture('{}/LHC_H2_&_O2_leak.png'.format(image_folder_name), column2_x, row2_y,width=width, height=height)
pic.click_action.hyperlink.address = '{}/LHC_H2_&_O2_leak.html'.format(image_folder_name)

# place the LWC Pressure_&_Conductivity chart
pic = slide.shapes.add_picture('{}/LHC_Pressure_&_Conductivity_Chart.png'.format(image_folder_name), column2_x, row1_y,width=width, height=height)
pic.click_action.hyperlink.address = '{}/Steady_State_Chart.html'.format(image_folder_name)

slide = prs.slides[3]
height = Inches(3.32)
width = Inches(5.1)
row1_y = Inches(0.59)
row2_y = Inches(4.18)
column1_x = Inches(0.48)
column2_x = Inches(7.15)

# place the LHC_Avg_Stack_Voltage_chart
pic = slide.shapes.add_picture('{}/LHC_Avg_Stack_Voltage_&_Current_Density.png'.format(image_folder_name), column2_x, row1_y,width=width, height=height)
pic.click_action.hyperlink.address = '{}/LHC_Avg_Stack_Voltage_&_Current_Density.html'.format(image_folder_name)

# place the LHC_Max_Stack_Voltage chart
pic = slide.shapes.add_picture('{}/LHC_Max_Stack_Voltage_&_Current_Density.png'.format(image_folder_name), column1_x, row1_y,width=width, height=height)
pic.click_action.hyperlink.address = '{}/LHC_Max_Stack_Voltage_&_Current_Density.html'.format(image_folder_name)

# place the LHC_Min_Stack_Voltage chart
pic = slide.shapes.add_picture('{}/LHC_Min_Stack_Voltage_&_Current_Density.png'.format(image_folder_name), column1_x, row2_y,width=width, height=height)
pic.click_action.hyperlink.address = '{}/LHC_Min_Stack_Voltage_&_Current_Density.html'.format(image_folder_name)

# place the Current Density Chart
pic = slide.shapes.add_picture('{}/Current_Density.png'.format(image_folder_name), column2_x, row2_y,width=width, height=height)
pic.click_action.hyperlink.address = '{}/Current_Density.html'.format(image_folder_name)

slide = prs.slides[4]
height = Inches(4.72)
width = Inches(6.29)
row1_y = Inches(1.25)
column1_x = Inches(0)
column2_x = Inches(7.04)

# place the Stack_ASR_Chart
pic = slide.shapes.add_picture('{}/Stack_ASR_Chart.png'.format(image_folder_name), column1_x, row1_y,width=width, height=height)
pic.click_action.hyperlink.address = '{}/Stack_ASR_Chart.html'.format(image_folder_name)

# place the Stack_ASR(max)_Chart
pic = slide.shapes.add_picture('{}/Stack_ASR(max)_Chart.png'.format(image_folder_name), column2_x, row1_y,width=width, height=height)
pic.click_action.hyperlink.address = '{}/Stack_ASR(max)_Chart.html'.format(image_folder_name)

slide = prs.slides[5]
height = Inches(4.72)
width = Inches(6.29)
row1_y = Inches(1.25)
column1_x = Inches(0)
column2_x = Inches(7.04)

height_1 = Inches(1.16)
width_1 = Inches(8.79)
row1_y_1 = Inches(6.15)
column1_x_1 = Inches(1.97)

# Check if the image files exist

# pol_curve_file = '{}/{}_pol_curve.png'.format(image_folder_name)
for m in ['U1','U2','L2','L1']:
    pol_curve_file = '{}/{}_pol_curve.png'.format(image_folder_name, m)
    if os.path.exists(pol_curve_file):
        pic = slide.shapes.add_picture(pol_curve_file, column2_x, row1_y, width=width, height=height)
        
avg_pol_curve_file = '{}/Pol_Curve_Avg_all_stacks.png'.format(image_folder_name)
minmax_cell_file = '{}/MinMax_cell.png'.format(image_folder_name)

# if os.path.exists(pol_curve_file):
#     pic = slide.shapes.add_picture(pol_curve_file, column2_x, row1_y, width=width, height=height)

if os.path.exists(avg_pol_curve_file):
    pic = slide.shapes.add_picture(avg_pol_curve_file, column1_x, row1_y, width=width, height=height)

if os.path.exists(minmax_cell_file):
    pic = slide.shapes.add_picture(minmax_cell_file, column1_x_1, row1_y_1, width=width_1, height=height_1)

slide = prs.slides[6]
height = Inches(3.55)
width = Inches(4.97)
row1_y = Inches(0.53)
row2_y = Inches(3.95)
column1_x = Inches(0.25)
column2_x = Inches(3.49)
column3_x = Inches(7.48)

# place the LWC Pressure 704 and PMP State chart
pic = slide.shapes.add_picture('{}/LWC_Pressure(PRT 704)_and_PMP_State.png'.format(image_folder_name), column1_x, row1_y,width=width, height=height)
pic.click_action.hyperlink.address = '{}/LWC_Pressure_and_PMP_State.html'.format(image_folder_name)

# place the LWC Pressure 705 and PMP State chart
pic = slide.shapes.add_picture('{}/LWC_Pressure(PRT 705)_and_PMP_State.png'.format(image_folder_name), column3_x, row1_y,width=width, height=height)
pic.click_action.hyperlink.address = '{}/LWC_Pressure_and_PMP_State.html'.format(image_folder_name)

# place the lWC conductivity chart
pic = slide.shapes.add_picture('{}/LWC_Conductivity.png'.format(image_folder_name), column2_x, row2_y,width=width, height=height)
pic.click_action.hyperlink.address = '{}/LWC_Conductivity.html'.format(image_folder_name)

# save the PowerPoint file with the customized name
prs.save('{}.pptx'.format(ppt_fn))

print("Presentation is published")

Presentation is published
